AInewsbot.ipynb

- Automate collecting daily AI news
- Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
- Save HTML of each URL in htmldata directory
- Extract URLs from all files, create a pandas dataframe with url, title, src
- Use ChatGPT to filter only AI-related headlines by sending a prompt and formatted table of headlines
- Use SQLite to filter headlines previously seen 
- OPENAI_API_KEY should be in the environment or in a .env file
  
Alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


1. initialize
2. fetch web pages
3. parse news story urls from web pages
4. filter headlines by relevance, not previously seen
5. perform topic analysis on headlines, and ordering by topic
6. summarize individual pages as bullet points
7. from bullet points, extract top 10 most common themes and stories of the day in order of importance
8. topic analysis of bullet points, categorize bullet points as belonging to particular themes
9. for each theme, make a summary and links. Here we want to iterate to improve summaries per specific criteria.
10. combine themes and send.

In [89]:
# import sys
# del sys.modules['ainb_const']


In [90]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json
import pickle
from collections import Counter

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# import bs4
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin, urlparse
import trafilatura

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from openai import OpenAI

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE)
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article, 
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file, 
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory, 
                            process_url_queue_factory, DRIVERS)
from ainb_llm import paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries, fetch_openai_summary, trunc_tokens


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


# Initialize

In [3]:
# OpenAI API module
client = OpenAI()

# Or can use REST API directly
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}


In [4]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources from {SOURCECONFIG}")

# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

log(f"Mapped {len(sources_reverse)} source page titles to sources")


2024-07-11 10:05:12,262 - AInewsbot - INFO - Load 17 sources from sources.yaml
2024-07-11 10:05:12,262 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-07-11 10:05:12,262 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology - Bloomberg.html
2024-07-11 10:05:12,263 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-07-11 10:05:12,263 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-07-11 10:05:12,263 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-07-11 10:0

20

In [5]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/',
  'sourcename': 'Ars Technica'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology',
  'sourcename': 'Bloomberg Tech'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech',
  'sourcename': 'Business Insider'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology',
  'sourcename': 'FT Tech'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=e

In [6]:
sources_reverse


{'Ars Technica': 'Ars Technica',
 'Bloomberg Technology - Bloomberg': 'Bloomberg Tech',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [7]:
# determine files already in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]
# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))

2024-07-11 10:05:12,296 - AInewsbot - INFO - 17
2024-07-11 10:05:12,297 - AInewsbot - INFO - htmldata/Technology - The Washington Post (07_11_2024 07_59_57 AM).html
2024-07-11 10:05:12,298 - AInewsbot - INFO - htmldata/Technology - WSJ.com (07_11_2024 07_59_51 AM).html
2024-07-11 10:05:12,298 - AInewsbot - INFO - htmldata/AI News _ VentureBeat (07_11_2024 07_59_47 AM).html
2024-07-11 10:05:12,298 - AInewsbot - INFO - htmldata/Discover and Add New Feedly AI Feeds (07_11_2024 07_59_44 AM).html
2024-07-11 10:05:12,298 - AInewsbot - INFO - htmldata/Artificial Intelligence - The Verge (07_11_2024 07_59_40 AM).html
2024-07-11 10:05:12,299 - AInewsbot - INFO - htmldata/top scoring links _ multi (07_11_2024 07_59_37 AM).html
2024-07-11 10:05:12,299 - AInewsbot - INFO - htmldata/The Register_ Enterprise Technology News and Analysis (07_11_2024 07_59_28 AM).html
2024-07-11 10:05:12,299 - AInewsbot - INFO - htmldata/Techmeme (07_11_2024 07_59_18 AM).html
2024-07-11 10:05:12,300 - AInewsbot - INFO

# Fetch and save source pages

In [8]:
# Fetch HTML files from sources

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
num_browsers = 3
log(f"Saving HTML files using {num_browsers} browsers")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
callable = process_source_queue_factory(queue)

saved_pages = launch_drivers(num_browsers, callable)


2024-07-11 10:05:12,358 - AInewsbot - INFO - Saving HTML files using 3 browsers
2024-07-11 10:05:12,369 - AInewsbot - INFO - get_driver - 32228 Initializing webdriver
2024-07-11 10:05:12,370 - AInewsbot - INFO - get_driver - 32228 Initializing webdriver
2024-07-11 10:05:12,370 - AInewsbot - INFO - get_driver - 32228 Initializing webdriver
2024-07-11 10:05:30,433 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-11 10:05:30,433 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-11 10:05:30,433 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-11 10:05:30,434 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-11 10:05:30,434 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-11 10:05:30,435 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-11 10:06:37,945 - AInewsbot - INFO - get_driver - Initialized webdriver
2024-07-11 10:06:37,965 - AInewsbot - INFO - get_dr

2024-07-11 10:07:23,901 - AInewsbot - INFO - get_files(Google News - Technology - Artificial intelligence) - Saving Google News - Technology - Artificial intelligence (07_11_2024 10_07_23 AM).html as utf-8
2024-07-11 10:07:23,916 - AInewsbot - INFO - Processing NYT Tech
2024-07-11 10:07:23,916 - AInewsbot - INFO - get_files(Technology - The New York Times) - starting get_files https://www.nytimes.com/section/technology
2024-07-11 10:07:31,086 - AInewsbot - INFO - get_files(Discover and Add New Feedly AI Feeds) - Loading additional infinite scroll items
2024-07-11 10:07:32,103 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrom

2024-07-11 10:08:18,207 - AInewsbot - INFO - get_files(Technology - WSJ.com) - Saving Technology - WSJ.com (07_11_2024 10_08_18 AM).html as utf-8
2024-07-11 10:08:18,208 - AInewsbot - INFO - Quit webdriver
2024-07-11 10:08:22,715 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-07-11 10:08:22,716 - AInewsbot - INFO - get_files(Technology - The Washington Post) - Saving Technology - The Washington Post (07_11_2024 10_08_22 AM).html as utf-8
2024-07-11 10:08:22,719 - AInewsbot - INFO - Quit webdriver


In [9]:
log(f"Saved {len(saved_pages)} pages")

print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    log("{sourcename} -> {page}")
    

2024-07-11 10:08:25,131 - AInewsbot - INFO - Saved 17 pages
2024-07-11 10:08:25,136 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,136 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,137 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,137 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,137 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,137 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,137 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,137 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,138 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,138 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,138 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,138 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,138 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-11 10:08:25,140 - AInewsbot - INFO - {sourcename

17


# Extract news URLs from saved pages

In [10]:
# Parse news URLs and titles from downloaded HTML files
log("Parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    log(sourcename +' -> ' + filename)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df.head()


2024-07-11 10:08:25,193 - AInewsbot - INFO - Parsing html files
2024-07-11 10:08:25,194 - AInewsbot - INFO - Business Insider -> htmldata/Tech - Business Insider (07_11_2024 10_06_49 AM).html
2024-07-11 10:08:25,194 - AInewsbot - INFO - parse loop - Business Insider
2024-07-11 10:08:25,238 - AInewsbot - INFO - parse_file - found 310 raw links
2024-07-11 10:08:25,244 - AInewsbot - INFO - parse_file - found 52 filtered links
2024-07-11 10:08:25,244 - AInewsbot - INFO - parse loop - 52 links found
2024-07-11 10:08:25,244 - AInewsbot - INFO - FT Tech -> htmldata/Technology (07_11_2024 10_07_00 AM).html
2024-07-11 10:08:25,245 - AInewsbot - INFO - parse loop - FT Tech
2024-07-11 10:08:25,282 - AInewsbot - INFO - parse_file - found 463 raw links
2024-07-11 10:08:25,287 - AInewsbot - INFO - parse_file - found 107 filtered links
2024-07-11 10:08:25,287 - AInewsbot - INFO - parse loop - 107 links found
2024-07-11 10:08:25,288 - AInewsbot - INFO - Hacker News -> htmldata/Hacker News Page 1 (07_1

,id,src,title,url
0,0,Ars Technica,Microsoft asks many Game Pass subscribers to p...,https://arstechnica.com/gaming/2024/07/microso...
1,1,Ars Technica,"To help with climate change, carbon capture wi...",https://arstechnica.com/science/2024/07/to-hel...
2,2,Ars Technica,Oregon county seeks to hold fossil fuel compan...,https://arstechnica.com/science/2024/07/oregon...
3,3,Ars Technica,New weight-loss and diabetes drugs linked to l...,https://arstechnica.com/science/2024/07/new-we...
4,4,Ars Technica,Nearby star cluster houses unusually large bla...,https://arstechnica.com/science/2024/07/nearby...


In [11]:
# # extracts all links from history where isAI=1
# # useful for training dimensionality reduction
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id"
# ai_history_df = pd.read_sql_query(query, conn)
# ai_history_df

In [12]:
# # clean up sqlite database if you want to rerun the job from a given point
# conn.execute(f"delete from news_articles where timestamp > '2024-07-08 19:15'")
# # conn.execute(f"delete from news_articles where id > 220230")
# # Committing the changes
# conn.commit()

# # Close the connection
# conn.close()


# Filter URLs to new AI headlines only

In [13]:
# filter urls we've already seen in previous runs and saved in SQLite
filtered_df = filter_unseen_urls_db(orig_df, before_date='2024-07-09 06:00:00')
len(filtered_df)

2024-07-11 10:08:26,763 - AInewsbot - INFO - Existing URLs: 125015
2024-07-11 10:08:26,786 - AInewsbot - INFO - New URLs: 794


794

In [14]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI
print(FILTER_PROMPT)



You will act as a research assistant to categorize news articles based on their relevance
to the topic of artificial intelligence (AI). You will closely read the title of each story
to determine if it is primarily about AI based on the semanting meaning of the title and
the keywords and entities mentioned. The input headlines and outptu classifications will
be formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]

Classification Criteria:
Classify each story based

In [15]:
# make pages that fit in a reasonably sized (MAXPAGELEN or MAX_INPUT_TOKENS) prompt
pages = paginate_df(filtered_df)
log(f"Paginated {len(pages)} pages")


2024-07-11 10:08:27,092 - AInewsbot - INFO - Paginated 16 pages


20

In [16]:
# use REST API directly. OpenAI python API doesn't support concurrent requests from a single client
# this runs fast with async aiohttp and on gpt-3.5 (15 seconds vs 2 minutes synchronously with gpt-4o)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

log("start classify")
enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT))
log("end classify")

enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))
enriched_df.head()


2024-07-11 10:08:27,096 - AInewsbot - INFO - start classify
2024-07-11 10:08:27,101 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,119 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,120 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,120 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,120 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,121 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,121 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,121 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,121 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,121 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,122 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,122 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,122 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,122 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,122 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:08:27,123 - AInewsbot - IN

891


,id,isAI
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False


In [17]:
# merge returned df with isAI column into original df on id column
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


,id,src,title,url,isAI,date
0,0,Ars Technica,Microsoft asks many Game Pass subscribers to p...,https://arstechnica.com/gaming/2024/07/microso...,False,2024-07-11
1,1,Ars Technica,"To help with climate change, carbon capture wi...",https://arstechnica.com/science/2024/07/to-hel...,False,2024-07-11
2,2,Ars Technica,Oregon county seeks to hold fossil fuel compan...,https://arstechnica.com/science/2024/07/oregon...,False,2024-07-11
3,3,Ars Technica,New weight-loss and diabetes drugs linked to l...,https://arstechnica.com/science/2024/07/new-we...,False,2024-07-11
4,4,Ars Technica,Nearby star cluster houses unusually large bla...,https://arstechnica.com/science/2024/07/nearby...,False,2024-07-11


In [18]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


2024-07-11 10:08:56,470 - AInewsbot - INFO - Unmatched response rows: 0
2024-07-11 10:08:56,473 - AInewsbot - INFO - Unmatched source rows: 1


20

In [19]:
# update SQLite database with all seen URLs
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

for row in merged_df.itertuples():
    insert_article(conn, cursor, row.src, row.title,
                   row.url, row.isAI, row.date)


In [20]:
# keep headlines that are related to AI
AIdf = merged_df.loc[merged_df["isAI"]==1] \
    .reset_index(drop=True)  \
    .reset_index()  \
    .drop(columns=["id"])  \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} AI headlines")
AIdf

2024-07-11 10:08:56,534 - AInewsbot - INFO - Found 234 AI headlines


,id,src,title,url,isAI,date
0,0,Ars Technica,"Could AIs become conscious? Right now, we have...",https://arstechnica.com/science/2024/07/could-...,True,2024-07-11
1,1,Ars Technica,"Three betas in, iOS 18 testers still can’t try...",https://arstechnica.com/gadgets/2024/07/three-...,True,2024-07-11
2,2,Ars Technica,"Galaxy Z Fold & Z Flip 6, Watch Ultra, and new...",https://arstechnica.com/gadgets/2024/07/the-ga...,True,2024-07-11
3,3,Ars Technica,How disinformation from a Russian AI spam farm...,https://arstechnica.com/ai/2024/07/how-disinfo...,True,2024-07-11
4,4,Bloomberg Tech,"Sequoia, Nvidia Back Startup Fireworks AI at $...",https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-11
...,...,...,...,...,...,...
229,229,VentureBeat,How LlamaIndex is ushering in the future of RA...,https://venturebeat.com/ai/how-llamaindex-is-u...,True,2024-07-11
230,230,VentureBeat,Anthropic Co-Founder: AI’s future lies in on-d...,https://venturebeat.com/ai/bespoke-software-on...,True,2024-07-11
231,231,WSJ Tech,It’s Time for AI to Start Making Money for Bus...,https://www.wsj.com/articles/its-time-for-ai-t...,True,2024-07-11
232,232,WaPo Tech,"Microsoft, Apple will not join OpenAI’s board ...",https://www.washingtonpost.com/technology/2024...,True,2024-07-11


In [21]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

def unicode_to_ascii(input_string):
    # Normalize the Unicode string to NFKD form
    normalized_string = unicodedata.normalize('NFKD', input_string)
    
    # Encode to ASCII bytes, ignoring characters that cannot be converted
    ascii_bytes = normalized_string.encode('ascii', 'ignore')
    
    # Convert bytes back to a string
    ascii_string = ascii_bytes.decode('ascii')
    
    return ascii_string

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [22]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index(drop=True) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI headlines")


2024-07-11 10:08:56,548 - AInewsbot - INFO - Found 212 unique AI headlines


20

In [23]:
# map google news headlines to redirect, kind of unnecessary
from urllib.parse import urlparse

redirect_dict = {}
for row in AIdf.itertuples():
    parsed_url = urlparse(row.url)
    netloc = parsed_url.netloc
    if netloc == 'news.google.com':
        print(netloc, end=" -> ")        
        response = requests.get(row.url, allow_redirects=False)
        # The URL to which it would have redirected
        redirect_url = response.headers.get('Location')
        redirect_dict[row.url] = redirect_url
        parsed_url2 = urlparse(redirect_url)
        netloc2 = parsed_url2.netloc
        if netloc2 == 'news.google.com':
            print(netloc2, end=" -> ")
            response = requests.get(redirect_url, allow_redirects=False)
        # The URL to which it would have redirected
            redirect_url = response.headers.get('Location')
            redirect_dict[row.url] = redirect_url
        print(redirect_url)

AIdf['url'] = AIdf['url'].apply(lambda url: redirect_dict.get(url, url))


news.google.com -> news.google.com -> https://finance.yahoo.com/news/2-popular-artificial-intelligence-ai-081200055.html
news.google.com -> news.google.com -> https://investorplace.com/2024/07/3-bitcoin-mining-stocks-to-buy-if-youre-betting-big-on-ai/
news.google.com -> news.google.com -> https://www.chronicle.com/article/a-professors-digital-mini-me
news.google.com -> news.google.com -> https://www.wired.com/story/ai-energy-demands-water-impact-internet-hyper-consumption-era/
news.google.com -> news.google.com -> https://www.wfla.com/news/florida/ai-automated-ammo-vending-machines-may-come-to-florida-grocery-stores-soon/
news.google.com -> news.google.com -> https://www.cbsnews.com/news/ai-generated-jokes-funnier-than-humans-university-of-southern-california-study/
news.google.com -> news.google.com -> https://phys.org/news/2024-07-ai-powered-atlas-reveals-proteins.html
news.google.com -> news.google.com -> https://www.insidehighered.com/news/tech-innovation/artificial-intelligence/20

news.google.com -> news.google.com -> https://www.euronews.com/culture/2024/07/09/meet-kenza-layli-from-morocco-the-winner-of-the-worlds-first-miss-ai-beauty-pageant
news.google.com -> news.google.com -> https://www.tomsguide.com/ai/meet-odyssey-ai-video-thats-fit-for-hollywood
news.google.com -> news.google.com -> https://timesofindia.indiatimes.com/technology/social/meet-the-winner-of-miss-ai-global-awards-and-indias-zara-shatavari-who-made-it-to-the-top-10/articleshow/111659113.cms
news.google.com -> news.google.com -> https://engineering.fb.com/2024/07/10/data-infrastructure/machine-learning-ml-prediction-robustness-meta/
news.google.com -> news.google.com -> https://www.extremetech.com/internet/microsofts-ai-speech-generator-is-too-realistic-to-release-researchers
news.google.com -> news.google.com -> https://www.wsj.com/tech/ai/microsoft-withdraws-from-openais-board-amid-antitrust-scrutiny-aab6ff1e
news.google.com -> news.google.com -> https://decrypt.co/238419/microsoft-ai-voice

In [24]:
AIdf

,id,src,title,url,isAI,date
0,0,HackerNoon,$4 Trillion Appears Inevitable as Nvidia Remai...,https://hackernoon.com/$4-trillion-appears-ine...,True,2024-07-11
1,1,Google News,2 Popular Artificial Intelligence (AI) Stocks ...,https://finance.yahoo.com/news/2-popular-artif...,True,2024-07-11
2,2,Google News,3 Bitcoin Mining Stocks to Buy If Youre Bettin...,https://investorplace.com/2024/07/3-bitcoin-mi...,True,2024-07-11
3,3,Google News,A College Is Tapping AI Teaching Assistants. W...,https://www.chronicle.com/article/a-professors...,True,2024-07-11
4,4,Google News,AI's Energy Demands Are Out of Control. Welcom...,https://www.wired.com/story/ai-energy-demands-...,True,2024-07-11
...,...,...,...,...,...,...
207,207,Google News,Women leaders assert: Generative AI needs huma...,https://venturebeat.com/ai/making-generative-a...,True,2024-07-11
208,208,VentureBeat,Writer drops mind-blowing AI update: RAG on st...,https://venturebeat.com/ai/writer-drops-mind-b...,True,2024-07-11
209,209,Business Insider,You can't use Canva's AI to create political f...,https://www.businessinsider.com/canva-ai-rules...,True,2024-07-11
210,210,Google News,eGrowcery launches AI recipes,https://www.supermarketnews.com/news/egrowcery...,True,2024-07-11


# Topic analysis
Here we are trying to identify the top topics of the day, to help make a nice summary. 

1st approach - do dimensionality reduction on the headline embeddings with UMAP and cluster with DBSCAN.

2nd approach
 - extract topics from headline using a prompt
 - human canonicalizes topics
 - assign headlines to topics using a prompt
 
 The final summary is pretty inconsistent, would be nice to give chatgpt a prompt that would say, summarize these bullet points using this categorization.
 

In [25]:
# attempt to extract top topics 
print(TOPIC_PROMPT)



You will act as a research assistant to extract topics from news headlines. You will extract topics, entities,
and keywords from news headlines formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 105,'title': "Former Microsoft CEO Steve Ballmer is now just as rich as his former boss Bill Gates. Here's how he spends his billions."},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
]

Output Specification:
You will return a JSON object with the field 'topics' containing a flat list of classification results.
For each headline input, your output will be a JSON object contain

In [26]:
# get topics
pages = paginate_df(AIdf)

# apply this prompt to AI headlines
log("start topic extraction")
response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT))
log("end topic extraction")

topic_df = pd.DataFrame(response)
print(len(topic_df))
topic_df.head()


2024-07-11 10:09:25,612 - AInewsbot - INFO - start topic extraction
2024-07-11 10:09:25,614 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:09:25,615 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:09:25,615 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:09:25,615 - AInewsbot - INFO - sent 50 items 
2024-07-11 10:09:25,616 - AInewsbot - INFO - sent 12 items 
2024-07-11 10:09:30,834 - AInewsbot - INFO - got dict with 12 items 
2024-07-11 10:09:41,964 - AInewsbot - INFO - got dict with 50 items 
2024-07-11 10:09:42,030 - AInewsbot - INFO - got dict with 50 items 
2024-07-11 10:09:47,990 - AInewsbot - INFO - got dict with 50 items 
2024-07-11 10:09:48,484 - AInewsbot - INFO - got dict with 50 items 
2024-07-11 10:09:48,487 - AInewsbot - INFO - Processed 212 responses.
2024-07-11 10:09:48,491 - AInewsbot - INFO - end topic extraction


212


,id,topics
0,0,"[Nvidia, Generative AI, technology, stock market]"
1,1,"[Artificial Intelligence, stocks, Wall Street,..."
2,2,"[Bitcoin, mining stocks, investment, AI]"
3,3,"[AI, teaching assistants, education]"
4,4,"[AI, energy consumption, internet]"


In [27]:
all_topics = [item for row in topic_df.itertuples() for item in row.topics]
item_counts = Counter(all_topics)
for x in item_counts.most_common():
    print(x)
    

('AI', 64)
('technology', 20)
('Microsoft', 20)
('OpenAI', 18)
('Samsung', 12)
('Apple', 11)
('Nvidia', 10)
('investment', 7)
('AMD', 7)
('Silo AI', 7)
('acquisition', 7)
('Elon Musk', 7)
('generative AI', 6)
('China', 6)
('board', 6)
('Generative AI', 5)
('automation', 5)
('funding', 5)
('Index Ventures', 4)
('future', 4)
('disinformation', 4)
('Intuit', 4)
('antitrust scrutiny', 4)
('artificial intelligence', 4)
('Artificial Intelligence', 3)
('Wall Street', 3)
('competition', 3)
('AWS', 3)
('marketing', 3)
('Windows 10', 3)
('layoffs', 3)
('Meta', 3)
('regulatory scrutiny', 3)
('Oracle', 3)
('medical condition', 3)
('Galaxy Ring', 3)
('Russia', 3)
('stock market', 2)
('Bitcoin', 2)
('productivity', 2)
('risks', 2)
('human parity', 2)
('scientists', 2)
('App Studio', 2)
('enterprise apps', 2)
('API', 2)
('Trump', 2)
('crypto', 2)
('Anthropic', 2)
('Claude 3.5 Sonnet', 2)
('test cases', 2)
('developers', 2)
('entertainment', 2)
('iPhone', 2)
('Artificial intelligence', 2)
('Biden', 2)

In [28]:
# evergreen topics to hopefully map healdines to canonical standardized topics
# review extracted topics and add
CANONICAL_TOPICS = [
    "Policy and regulation",
    "AI economic impacts",
    "Robots",
    "Autonomous vehicles",
    "AI job market",
    "LLMs",
    "Healthcare",
    "Fintech",
    "Education",
    "Entertainment",
    "Startup funding",
    "IPOs",
    "Ethical issues",
    "Legal issues",
    "Cybersecurity",
    "AI doom",
    'Stocks',
    'Climate',
    'Scams',
    'Privacy',
    'Intellectual Property',
    'Code assistants',
    'Customer service',
    'Reinforcement Learning',
    'Open Source',
    'Language Models',
    'China',
    'Military',
    'Semiconductor chips',
    'Sustainability',
    'Agriculture',
    'Gen AI',
    'Testing',
    
    'Nvidia',
    'Google',
    'OpenAI',
    'Meta',
    'Apple',
    'Microsoft',
    'Salesforce',
    'Uber',
    'AMD',
    'Netflix',
    'Disney',
    'Amazon',
    'Cloudflare',
    'Anthropic',
    'Cohere',
    'Baidu',
    'Big Tech',
    'Samsung',
    'Tesla',
    
    'ChatGPT',
    'WhatsApp',
    'Gemini',
    'Claude',
    'Copilot',
    
    'Elon Musk',
    'Bill Gates',
    'Sam Altman',
    'Mustafa Suleyman',
    'Sundar Pichai',
    'Yann LeCun',
    'Geoffrey Hinton',
    'Mark Zuckerberg',
]

In [29]:
# you could try it with new cats or new cats + evergreen
# but probably look at new cats and human in the loop should add good new cats today to evergreen list
# new_cats = list(json.loads(response.choices[0].message.content).values())[0]
# categories = sorted(list(set(new_cats + evergreen)))
categories = sorted(CANONICAL_TOPICS)
categories


['AI doom',
 'AI economic impacts',
 'AI job market',
 'AMD',
 'Agriculture',
 'Amazon',
 'Anthropic',
 'Apple',
 'Autonomous vehicles',
 'Baidu',
 'Big Tech',
 'Bill Gates',
 'ChatGPT',
 'China',
 'Claude',
 'Climate',
 'Cloudflare',
 'Code assistants',
 'Cohere',
 'Copilot',
 'Customer service',
 'Cybersecurity',
 'Disney',
 'Education',
 'Elon Musk',
 'Entertainment',
 'Ethical issues',
 'Fintech',
 'Gemini',
 'Gen AI',
 'Geoffrey Hinton',
 'Google',
 'Healthcare',
 'IPOs',
 'Intellectual Property',
 'LLMs',
 'Language Models',
 'Legal issues',
 'Mark Zuckerberg',
 'Meta',
 'Microsoft',
 'Military',
 'Mustafa Suleyman',
 'Netflix',
 'Nvidia',
 'Open Source',
 'OpenAI',
 'Policy and regulation',
 'Privacy',
 'Reinforcement Learning',
 'Robots',
 'Salesforce',
 'Sam Altman',
 'Samsung',
 'Scams',
 'Semiconductor chips',
 'Startup funding',
 'Stocks',
 'Sundar Pichai',
 'Sustainability',
 'Tesla',
 'Testing',
 'Uber',
 'WhatsApp',
 'Yann LeCun']

In [30]:
async def categorize_story(headline, categories, session, 
                           model=LOWCOST_MODEL,
                           temperature=0.5,
                           max_retries=MAX_RETRIES):
    
    retlist = []
    if type(categories) is not list:
        categories = [categories]
    for topic in categories:
        cat_prompt = f"""You are a news topic categorizaton assistant. I will provide a headline 
and a topic. You will respond with a JSON object {{'response': 1}} if the news headline matches 
the news topic and {{'response': 0}} if it does not. Check carefully and only return {{'response': 1}}
if the headline closely matches the topic. If the headline is not a close match or if unsure, 
return {{'response': 0}}
Headline:
{headline}
Topic:
{topic}
"""
        for i in range(max_retries):
            try:
                messages=[
                          {"role": "user", "content": cat_prompt
                          }]

                payload = {"model":  model,
                           "response_format": {"type": "json_object"},
                           "messages": messages,
                           "temperature": temperature
                           }
                response = await fetch_openai(session, payload)
                response_dict = json.loads(response["choices"][0]["message"]["content"])
                response_val = response_dict['response']
                if response_val == 1:
                    retlist.append(topic)
                break
            except Exception as exc:
                log(f"Error: {exc}")

            
    return retlist
        

h = "Utility stocks are Wall Streets secret backdoor to AI"
catdict = dict()

async with aiohttp.ClientSession() as session:
    for i, row in enumerate(AIdf.itertuples()):
        tasks = []
        log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
        h = row.title
        log(h)
        for c in categories:
            task = asyncio.create_task(categorize_story(h, c, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        catdict[row.id] = [item for sublist in responses for item in sublist]
        log(str(catdict[row.id]))
        


2024-07-11 10:09:48,524 - AInewsbot - INFO - Categorizing headline 1 of 212
2024-07-11 10:09:48,524 - AInewsbot - INFO - $4 Trillion Appears Inevitable as Nvidia Remains the Star of the Generative AI Boom
2024-07-11 10:09:50,157 - AInewsbot - INFO - ['AI economic impacts', 'Gen AI', 'Nvidia', 'Semiconductor chips', 'Stocks']
2024-07-11 10:09:50,159 - AInewsbot - INFO - Categorizing headline 2 of 212
2024-07-11 10:09:50,159 - AInewsbot - INFO - 2 Popular Artificial Intelligence (AI) Stocks to Sell Before They Plunge 64% and 67%, According to Certain Wall Street Analysts
2024-07-11 10:09:51,843 - AInewsbot - INFO - ['Stocks']
2024-07-11 10:09:51,845 - AInewsbot - INFO - Categorizing headline 3 of 212
2024-07-11 10:09:51,846 - AInewsbot - INFO - 3 Bitcoin Mining Stocks to Buy If Youre Betting Big on AI
2024-07-11 10:09:52,912 - AInewsbot - INFO - []
2024-07-11 10:09:52,913 - AInewsbot - INFO - Categorizing headline 4 of 212
2024-07-11 10:09:52,914 - AInewsbot - INFO - A College Is Tapping

2024-07-11 10:10:34,042 - AInewsbot - INFO - ['AI economic impacts', 'Gen AI']
2024-07-11 10:10:34,043 - AInewsbot - INFO - Categorizing headline 30 of 212
2024-07-11 10:10:34,044 - AInewsbot - INFO - A former OpenAI safety employee said he quit because the company's leaders were 'building the Titanic' and wanted 'newer, shinier' things to sell
2024-07-11 10:10:35,747 - AInewsbot - INFO - ['AI doom', 'Big Tech', 'Ethical issues', 'OpenAI']
2024-07-11 10:10:35,748 - AInewsbot - INFO - Categorizing headline 31 of 212
2024-07-11 10:10:35,749 - AInewsbot - INFO - Aitomatics SemiKong uses AI to reshape chipmaking processes
2024-07-11 10:10:37,144 - AInewsbot - INFO - ['AI economic impacts', 'Big Tech', 'Semiconductor chips']
2024-07-11 10:10:37,145 - AInewsbot - INFO - Categorizing headline 32 of 212
2024-07-11 10:10:37,146 - AInewsbot - INFO - Amazon announces incremental AI refinements to fend off rivals
2024-07-11 10:10:37,924 - AInewsbot - INFO - ['AI economic impacts', 'AI job market',

2024-07-11 10:11:07,289 - AInewsbot - INFO - ['AI economic impacts', 'China', 'Open Source']
2024-07-11 10:11:07,291 - AInewsbot - INFO - Categorizing headline 57 of 212
2024-07-11 10:11:07,291 - AInewsbot - INFO - Citi Research analysts: Take profits in AI high-fliers (NASDAQ:AIQ)
2024-07-11 10:11:08,118 - AInewsbot - INFO - ['Stocks']
2024-07-11 10:11:08,120 - AInewsbot - INFO - Categorizing headline 58 of 212
2024-07-11 10:11:08,120 - AInewsbot - INFO - Citi names undervalued AI stocks poised for rerating
2024-07-11 10:11:09,097 - AInewsbot - INFO - ['Fintech', 'Stocks']
2024-07-11 10:11:09,099 - AInewsbot - INFO - Categorizing headline 59 of 212
2024-07-11 10:11:09,100 - AInewsbot - INFO - Could AIs become conscious? Right now, we have no way to tell.
2024-07-11 10:11:10,400 - AInewsbot - INFO - ['Ethical issues', 'Gen AI']
2024-07-11 10:11:10,401 - AInewsbot - INFO - Categorizing headline 60 of 212
2024-07-11 10:11:10,401 - AInewsbot - INFO - Cybersecurity and AI: Meetings and Ins

2024-07-11 10:11:40,850 - AInewsbot - INFO - Categorizing headline 86 of 212
2024-07-11 10:11:40,851 - AInewsbot - INFO - Habib at VB Transform: Writers vision for full stack AI
2024-07-11 10:11:41,686 - AInewsbot - INFO - []
2024-07-11 10:11:41,687 - AInewsbot - INFO - Categorizing headline 87 of 212
2024-07-11 10:11:41,687 - AInewsbot - INFO - Hanging onto Windows 10 to avoid Microsofts latest brainwaves with Copilot AI? Ive got bad news for you
2024-07-11 10:11:42,673 - AInewsbot - INFO - ['AI doom', 'Big Tech', 'Code assistants', 'Copilot', 'Language Models', 'Microsoft', 'Privacy', 'Robots']
2024-07-11 10:11:42,675 - AInewsbot - INFO - Categorizing headline 88 of 212
2024-07-11 10:11:42,676 - AInewsbot - INFO - Has Artificial Intelligence Co-opted the Sparkle Emoji?
2024-07-11 10:11:43,288 - AInewsbot - INFO - []
2024-07-11 10:11:43,290 - AInewsbot - INFO - Categorizing headline 89 of 212
2024-07-11 10:11:43,291 - AInewsbot - INFO - Hayden AI, which provides vision AI tools for ci

2024-07-11 10:12:13,468 - AInewsbot - INFO - Categorizing headline 115 of 212
2024-07-11 10:12:13,469 - AInewsbot - INFO - Meet Kenza Layli - the winner of the first Miss AI beauty pageant
2024-07-11 10:12:14,468 - AInewsbot - INFO - ['Entertainment', 'Gen AI']
2024-07-11 10:12:14,470 - AInewsbot - INFO - Categorizing headline 116 of 212
2024-07-11 10:12:14,471 - AInewsbot - INFO - Meet Odyssey  AI video thats fit for Hollywood
2024-07-11 10:12:15,234 - AInewsbot - INFO - ['Entertainment', 'Robots']
2024-07-11 10:12:15,235 - AInewsbot - INFO - Categorizing headline 117 of 212
2024-07-11 10:12:15,236 - AInewsbot - INFO - Meet the winner of Miss AI global awards; and India's Zara Shatavari who made it to the top 10
2024-07-11 10:12:15,845 - AInewsbot - INFO - []
2024-07-11 10:12:15,846 - AInewsbot - INFO - Categorizing headline 118 of 212
2024-07-11 10:12:15,846 - AInewsbot - INFO - Metas approach to machine learning prediction robustness
2024-07-11 10:12:17,530 - AInewsbot - INFO - ['Me

2024-07-11 10:12:41,870 - AInewsbot - INFO - ['Ethical issues', 'LLMs', 'OpenAI', 'Testing']
2024-07-11 10:12:41,871 - AInewsbot - INFO - Categorizing headline 141 of 212
2024-07-11 10:12:41,873 - AInewsbot - INFO - OpenAI board shake-up: Microsoft out, Apple backs away amid AI partnership scrutiny
2024-07-11 10:12:42,504 - AInewsbot - INFO - ['Big Tech', 'Ethical issues', 'Microsoft', 'OpenAI']
2024-07-11 10:12:42,506 - AInewsbot - INFO - Categorizing headline 142 of 212
2024-07-11 10:12:42,507 - AInewsbot - INFO - Oracle Stock Falls On Report That Talks Are Off For Cloud Deal With Elon Musk's AI Startup
2024-07-11 10:12:43,499 - AInewsbot - INFO - ['Big Tech', 'Copilot', 'Elon Musk', 'OpenAI', 'Stocks']
2024-07-11 10:12:43,501 - AInewsbot - INFO - Categorizing headline 143 of 212
2024-07-11 10:12:43,502 - AInewsbot - INFO - Oracle Stock Is Down on AI News. Palantir Shouldnt Be Overlooked.
2024-07-11 10:12:44,257 - AInewsbot - INFO - ['Stocks']
2024-07-11 10:12:44,258 - AInewsbot - IN

2024-07-11 10:13:11,110 - AInewsbot - INFO - ['AI economic impacts']
2024-07-11 10:13:11,111 - AInewsbot - INFO - Categorizing headline 169 of 212
2024-07-11 10:13:11,112 - AInewsbot - INFO - Skild AI, which is building a foundational model for robotics, raised a $300M Series A at a $1.5B valuation led by Lightspeed, SoftBank, Coatue, and Jeff Bezos
2024-07-11 10:13:12,416 - AInewsbot - INFO - ['Amazon', 'Big Tech', 'Robots', 'Startup funding']
2024-07-11 10:13:12,417 - AInewsbot - INFO - Categorizing headline 170 of 212
2024-07-11 10:13:12,418 - AInewsbot - INFO - Solving everything wrong with Large Language Models
2024-07-11 10:13:13,336 - AInewsbot - INFO - ['ChatGPT', 'Cohere', 'Ethical issues', 'LLMs', 'Language Models', 'OpenAI']
2024-07-11 10:13:13,338 - AInewsbot - INFO - Categorizing headline 171 of 212
2024-07-11 10:13:13,339 - AInewsbot - INFO - SoundHound Chat AI Launches in Peugeot, Opel, and Vauxhall Vehicles Throughout Europe
2024-07-11 10:13:14,288 - AInewsbot - INFO - 

2024-07-11 10:13:36,963 - AInewsbot - INFO - WWT CEO On 'Unhappy' Broadcom VMware Customers Seeking Alternatives And WWT 'Tripling' AI Initiatives
2024-07-11 10:13:38,615 - AInewsbot - INFO - []
2024-07-11 10:13:38,616 - AInewsbot - INFO - Categorizing headline 197 of 212
2024-07-11 10:13:38,616 - AInewsbot - INFO - Warnings About an AI Bubble Are Growing. When Could It Burst?
2024-07-11 10:13:40,286 - AInewsbot - INFO - ['AI doom', 'AI economic impacts', 'OpenAI']
2024-07-11 10:13:40,288 - AInewsbot - INFO - Categorizing headline 198 of 212
2024-07-11 10:13:40,289 - AInewsbot - INFO - We Cannot Cede Control of Weapons to Artificial Intelligence
2024-07-11 10:13:41,213 - AInewsbot - INFO - ['AI doom', 'Elon Musk', 'Ethical issues', 'Gen AI', 'Legal issues', 'Military', 'OpenAI', 'Policy and regulation', 'Robots']
2024-07-11 10:13:41,216 - AInewsbot - INFO - Categorizing headline 199 of 212
2024-07-11 10:13:41,217 - AInewsbot - INFO - Welcome to the new Fightertown USA: Inside San Diego

In [31]:
catdict

{0: ['AI economic impacts',
  'Gen AI',
  'Nvidia',
  'Semiconductor chips',
  'Stocks'],
 1: ['Stocks'],
 2: [],
 3: ['Education', 'Ethical issues', 'Gen AI', 'Language Models', 'Robots'],
 4: ['AI doom', 'AI economic impacts', 'Sustainability'],
 5: [],
 6: ['ChatGPT', 'Gen AI', 'Language Models', 'Robots'],
 7: ['Gen AI'],
 8: [],
 9: ['Meta', 'OpenAI', 'Reinforcement Learning', 'Robots'],
 10: ['Agriculture', 'Sustainability'],
 11: [],
 12: ['Google'],
 13: ['AI economic impacts', 'Semiconductor chips', 'Stocks'],
 14: ['AI economic impacts', 'AMD', 'Fintech', 'Nvidia', 'Stocks'],
 15: ['AI doom',
  'Big Tech',
  'Ethical issues',
  'Gen AI',
  'Language Models',
  'OpenAI',
  'Policy and regulation'],
 16: ['AI doom',
  'Ethical issues',
  'Gen AI',
  'Language Models',
  'OpenAI',
  'Policy and regulation'],
 17: ['Cohere',
  'Gen AI',
  'Geoffrey Hinton',
  'Language Models',
  'OpenAI',
  'Testing'],
 18: ['AI economic impacts', 'AMD', 'Stocks'],
 19: ['AI economic impacts', '

In [33]:
AIdf.head()

,id,src,title,url,isAI,date
0,0,HackerNoon,$4 Trillion Appears Inevitable as Nvidia Remai...,https://hackernoon.com/$4-trillion-appears-ine...,True,2024-07-11
1,1,Google News,2 Popular Artificial Intelligence (AI) Stocks ...,https://finance.yahoo.com/news/2-popular-artif...,True,2024-07-11
2,2,Google News,3 Bitcoin Mining Stocks to Buy If Youre Bettin...,https://investorplace.com/2024/07/3-bitcoin-mi...,True,2024-07-11
3,3,Google News,A College Is Tapping AI Teaching Assistants. W...,https://www.chronicle.com/article/a-professors...,True,2024-07-11
4,4,Google News,AI's Energy Demands Are Out of Control. Welcom...,https://www.wired.com/story/ai-energy-demands-...,True,2024-07-11


In [32]:
AIdf = AIdf.drop(columns=["assigned_topics"])

KeyError: "['assigned_topics'] not found in axis"

In [34]:
topic_df["assigned_topics"] = topic_df["id"].apply(lambda id: catdict.get(id, []))
topic_df

,id,topics,assigned_topics
0,0,"[Nvidia, Generative AI, technology, stock market]","[AI economic impacts, Gen AI, Nvidia, Semicond..."
1,1,"[Artificial Intelligence, stocks, Wall Street,...",[Stocks]
2,2,"[Bitcoin, mining stocks, investment, AI]",[]
3,3,"[AI, teaching assistants, education]","[Education, Ethical issues, Gen AI, Language M..."
4,4,"[AI, energy consumption, internet]","[AI doom, AI economic impacts, Sustainability]"
...,...,...,...
207,207,"[Generative AI, women leaders, human involveme...","[Ethical issues, Gen AI]"
208,208,"[AI update, RAG model, AI capacity, technology...","[Big Tech, ChatGPT, Cohere, Gen AI, LLMs, Lang..."
209,209,"[Canva, AI, political flyers, medical misinfor...","[Ethical issues, Legal issues, Policy and regu..."
210,210,"[eGrowcery, AI recipes, food industry, technol...",[Gen AI]


In [37]:
def clean_topics(row):
    topics = [x.title() for x in row.topics if x.lower() not in {"ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics]
    combined = sorted(list(set(topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    
    return ", ".join(combined)

topic_df["topic_str"] = topic_df.apply(clean_topics, axis=1)
topic_df

,id,topics,assigned_topics,clean_topics,topic_str
0,0,"[Nvidia, Generative AI, technology, stock market]","[AI economic impacts, Gen AI, Nvidia, Semicond...","AI Economic Impacts, Gen AI, Generative AI, Nv...","AI Economic Impacts, Gen AI, Generative AI, Nv..."
1,1,"[Artificial Intelligence, stocks, Wall Street,...",[Stocks],"Investment, Stocks, Wall Street","Investment, Stocks, Wall Street"
2,2,"[Bitcoin, mining stocks, investment, AI]",[],"Bitcoin, Investment, Mining Stocks","Bitcoin, Investment, Mining Stocks"
3,3,"[AI, teaching assistants, education]","[Education, Ethical issues, Gen AI, Language M...","Education, Ethical Issues, Gen AI, Language Mo...","Education, Ethical Issues, Gen AI, Language Mo..."
4,4,"[AI, energy consumption, internet]","[AI doom, AI economic impacts, Sustainability]","AI Doom, AI Economic Impacts, Energy Consumpti...","AI Doom, AI Economic Impacts, Energy Consumpti..."
...,...,...,...,...,...
207,207,"[Generative AI, women leaders, human involveme...","[Ethical issues, Gen AI]","Ethical Issues, Gen AI, Generative AI, Human I...","Ethical Issues, Gen AI, Generative AI, Human I..."
208,208,"[AI update, RAG model, AI capacity, technology...","[Big Tech, ChatGPT, Cohere, Gen AI, LLMs, Lang...","AI Capacity, AI Update, Big Tech, Chatgpt, Coh...","AI Capacity, AI Update, Big Tech, Chatgpt, Coh..."
209,209,"[Canva, AI, political flyers, medical misinfor...","[Ethical issues, Legal issues, Policy and regu...","Canva, Ethical Issues, Legal Issues, Medical M...","Canva, Ethical Issues, Legal Issues, Medical M..."
210,210,"[eGrowcery, AI recipes, food industry, technol...",[Gen AI],"AI Recipes, Egrowcery, Food Industry, Gen AI, ...","AI Recipes, Egrowcery, Food Industry, Gen AI, ..."


In [38]:
# merge returned df into original df
merged_df = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="outer")
merged_df['title_topic_str'] = merged_df.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)

merged_df


,id,src,title,url,isAI,date,topic_str,title_topic_str
0,0,HackerNoon,$4 Trillion Appears Inevitable as Nvidia Remai...,https://hackernoon.com/$4-trillion-appears-ine...,True,2024-07-11,"AI Economic Impacts, Gen AI, Generative AI, Nv...",$4 Trillion Appears Inevitable as Nvidia Remai...
1,1,Google News,2 Popular Artificial Intelligence (AI) Stocks ...,https://finance.yahoo.com/news/2-popular-artif...,True,2024-07-11,"Investment, Stocks, Wall Street",2 Popular Artificial Intelligence (AI) Stocks ...
2,2,Google News,3 Bitcoin Mining Stocks to Buy If Youre Bettin...,https://investorplace.com/2024/07/3-bitcoin-mi...,True,2024-07-11,"Bitcoin, Investment, Mining Stocks",3 Bitcoin Mining Stocks to Buy If Youre Bettin...
3,3,Google News,A College Is Tapping AI Teaching Assistants. W...,https://www.chronicle.com/article/a-professors...,True,2024-07-11,"Education, Ethical Issues, Gen AI, Language Mo...",A College Is Tapping AI Teaching Assistants. W...
4,4,Google News,AI's Energy Demands Are Out of Control. Welcom...,https://www.wired.com/story/ai-energy-demands-...,True,2024-07-11,"AI Doom, AI Economic Impacts, Energy Consumpti...",AI's Energy Demands Are Out of Control. Welcom...
...,...,...,...,...,...,...,...,...
207,207,Google News,Women leaders assert: Generative AI needs huma...,https://venturebeat.com/ai/making-generative-a...,True,2024-07-11,"Ethical Issues, Gen AI, Generative AI, Human I...",Women leaders assert: Generative AI needs huma...
208,208,VentureBeat,Writer drops mind-blowing AI update: RAG on st...,https://venturebeat.com/ai/writer-drops-mind-b...,True,2024-07-11,"AI Capacity, AI Update, Big Tech, Chatgpt, Coh...",Writer drops mind-blowing AI update: RAG on st...
209,209,Business Insider,You can't use Canva's AI to create political f...,https://www.businessinsider.com/canva-ai-rules...,True,2024-07-11,"Canva, Ethical Issues, Legal Issues, Medical M...",You can't use Canva's AI to create political f...
210,210,Google News,eGrowcery launches AI recipes,https://www.supermarketnews.com/news/egrowcery...,True,2024-07-11,"AI Recipes, Egrowcery, Food Industry, Gen AI, ...",eGrowcery launches AI recipes (Topics: AI Reci...


In [39]:
AIdf = merged_df

### Semantic sort

In [40]:
# use embeddings to sort headlines by semantical similarity
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf = AIdf.iloc[sorted_indices] \
    .reset_index(drop=True) \
    .reset_index() \
    .drop(columns=["id"]) \
    .rename(columns={'index': 'id'})

# sort embedding_df to match
embedding_df = embedding_df[sorted_indices]

AIdf


2024-07-11 10:30:53,185 - AInewsbot - INFO - Fetching embeddings for 212 headlines
2024-07-11 10:30:53,768 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


,id,src,title,url,isAI,date,topic_str,title_topic_str
0,0,Google News,"Samsung launches first-of-a-kind smart ring, w...",https://nypost.com/2024/07/10/business/samsung...,True,2024-07-11,"AI Features, Big Tech, Health Monitoring, Heal...","Samsung launches first-of-a-kind smart ring, w..."
1,1,Google News,Samsung's new Galaxy Ring will provide users w...,https://qz.com/samsung-galaxy-smart-ring-oura-...,True,2024-07-11,"AI-Powered Health Insights, Big Tech, Galaxy R...",Samsung's new Galaxy Ring will provide users w...
2,2,Business Insider,"Samsung unveils its new Galaxy Ring, an AI-bac...",https://www.businessinsider.com/guides/tech/sa...,True,2024-07-11,"Big Tech, Galaxy Ring, Oura Ring, Samsung","Samsung unveils its new Galaxy Ring, an AI-bac..."
3,3,Google News,Samsung Unveils Exclusive Galaxy Z Flip6 Olymp...,https://news.samsung.com/global/samsung-unveil...,True,2024-07-11,"Athletes, Big Tech, Galaxy AI, Galaxy Z Flip6,...",Samsung Unveils Exclusive Galaxy Z Flip6 Olymp...
4,4,Business Insider,"Samsung announces its latest smartwatches, inc...",https://www.businessinsider.com/guides/tech/sa...,True,2024-07-11,"Apple, Big Tech, Galaxy Watch Ultra, Samsung, ...","Samsung announces its latest smartwatches, inc..."
...,...,...,...,...,...,...,...,...
207,207,HackerNoon,$4 Trillion Appears Inevitable as Nvidia Remai...,https://hackernoon.com/$4-trillion-appears-ine...,True,2024-07-11,"AI Economic Impacts, Gen AI, Generative AI, Nv...",$4 Trillion Appears Inevitable as Nvidia Remai...
208,208,Google News,Goldman Sachs: $1tn to be spent on AI data cen...,https://www.datacenterdynamics.com/en/news/gol...,True,2024-07-11,"AI Economic Impacts, Big Tech, Chips, Data Cen...",Goldman Sachs: $1tn to be spent on AI data cen...
209,209,The Register,CyrusOne scores another $7.9B in debt financin...,https://www.theregister.com/2024/07/10/cyruson...,True,2024-07-11,"Cyrusone, Datacenter, Debt Financing, Expansio...",CyrusOne scores another $7.9B in debt financin...
210,210,Google News,Oracle Stock Falls On Report That Talks Are Of...,https://www.investors.com/news/technology/orac...,True,2024-07-11,"AI Startup, Big Tech, Cloud Deal, Copilot, Elo...",Oracle Stock Falls On Report That Talks Are Of...


In [41]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(AIdf[["title"]])
    

,title
0,"Samsung launches first-of-a-kind smart ring, watch with AI features to help monitor your health"
1,Samsung's new Galaxy Ring will provide users with AI-powered health insights
2,"Samsung unveils its new Galaxy Ring, an AI-backed competitor to the Oura Ring"
3,"Samsung Unveils Exclusive Galaxy Z Flip6 Olympic Edition, Powered by Galaxy AI, for Paris 2024 Athletes"
4,"Samsung announces its latest smartwatches, including a brand-new Galaxy Watch Ultra designed to take on Apple's high-end wearable"
5,"Samsung launches the Galaxy Z Fold 6 and Z Flip 6, its most durable foldable phones to date"
6,"Galaxy Z Fold & Z Flip 6, Watch Ultra, and new Ring are Samsungs AI carriers"
7,"Samsung Galaxy Unpacked News: Galaxy Ring, Galaxy Z Fold 6, Galaxy Watch Ultra and more"
8,"Analysis | Tech winners from Trumps 2024 platform: crypto, AI and Elon Musk"
9,"Tech winners from Trumps 2024 platform: crypto, AI and Elon Musk"


### Cluster with DBSCAN

In [42]:
# embedding_model = 'text-embedding-3-large'
# chunksize = 1000
# e_results = []
# for start in range(0, len(ai_history_df), chunksize):
#     tempdf = ai_history_df.iloc[start:start+chunksize]
#     templist = tempdf['title'].tolist()
#     log(f"Fetching embeddings for {len(templist)} headlines starting at row {start}")
#     response = client.embeddings.create(input=templist,
#                                         model=embedding_model)
#     e_results.append(pd.DataFrame([e.model_dump()['embedding'] for e in response.data]))


In [43]:
# historical_embedding_df = pd.concat(e_results)
# historical_embedding_df.shape


In [44]:
# historical_embedding_df.to_pickle('historical_embedding_df.pkl')


In [45]:
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])


2024-07-11 10:30:56,030 - AInewsbot - INFO - Fetching embeddings for 212 headlines
2024-07-11 10:30:56,955 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [46]:
# reducer = umap.UMAP(n_components=30)  # Reducing to 30 dimensions
# reduced_data = reducer.fit_transform(historical_embedding_df)
# with open('umap_model.pkl', 'wb') as file:
#     pickle.dump(reducer, file)

In [47]:
with open("umap_model.pkl", 'rb') as file:
    # Load the model from the file
    reducer = pickle.load(file)
    

In [48]:
reduced_data = reducer.transform(embedding_df)


In [49]:
np.isnan(reduced_data).any()


False

In [50]:
# Apply a Clustering Algorithm (e.g., K-Means)
kmeans = KMeans(n_clusters=20)  
clusters = kmeans.fit_predict(reduced_data)

# Evaluate the Clustering
silhouette_avg = silhouette_score(reduced_data, clusters)
print(f'Silhouette Score: {silhouette_avg}')

# Visualization with UMAP (optional)
# reducer_2d = umap.UMAP(n_components=2)  # Reducing to 2 dimensions for visualization
# reduced_data_2d = reducer_2d.fit_transform(embedding_df)

# plt.scatter(reduced_data_2d[:, 0], reduced_data_2d[:, 1], c=clusters, cmap='viridis', s=5)
# plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
# plt.title('UMAP Projection of the News Headlines Clusters')
# plt.show()


Silhouette Score: 0.4050346910953522


In [51]:
dbscan = DBSCAN(eps=0.5, min_samples=3)  # Adjust eps and min_samples as needed
clusters = dbscan.fit_predict(reduced_data)

AIdf['cluster'] = clusters

In [52]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    for i in range(30):
        tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title"]]
        if len(tmpdf) ==0:
            break
        display(tmpdf)
    


,id,title
0,0,"Samsung launches first-of-a-kind smart ring, watch with AI features to help monitor your health"
1,1,Samsung's new Galaxy Ring will provide users with AI-powered health insights
2,2,"Samsung unveils its new Galaxy Ring, an AI-backed competitor to the Oura Ring"
3,3,"Samsung Unveils Exclusive Galaxy Z Flip6 Olympic Edition, Powered by Galaxy AI, for Paris 2024 Athletes"
4,4,"Samsung announces its latest smartwatches, including a brand-new Galaxy Watch Ultra designed to take on Apple's high-end wearable"
5,5,"Samsung launches the Galaxy Z Fold 6 and Z Flip 6, its most durable foldable phones to date"
6,6,"Galaxy Z Fold & Z Flip 6, Watch Ultra, and new Ring are Samsungs AI carriers"
7,7,"Samsung Galaxy Unpacked News: Galaxy Ring, Galaxy Z Fold 6, Galaxy Watch Ultra and more"
21,21,Samsung's new lineup shows how AI is driving smartphone growth
22,22,Here's every AI feature coming to Samsung Galaxy foldable phones


,id,title
14,14,Biden Offers $1.7 Billion to Help Factories Build Electric Vehicles
16,16,FANUC America unveils new $110 million robotics and automation campus
17,17,Fanuc opens $110m automation campus in the US


,id,title
18,18,Apple seems super confident AI will make you want to buy a new iPhone
19,19,Apple seems super confident that AI will make you want to buy a new iPhone
31,31,"Three betas in, iOS 18 testers still cant try out Apple Intelligence features"
33,33,Apple TV+ Sunny looks at deadly AI after Apple Intelligence debut


,id,title
35,35,"Intuit decimates staff, hopes to hire same number in AI refocusWhat's that Lassie? The IRS hopes to offer free direct tax filing for everyone next year?Applications19 hrs|6"
36,36,Tax preparation company Intuit to lay off 1800 as part of an AI-focused reorganization plan
37,37,"Intuit is laying off 1,800 employees as AI leads to a strategic shift"
38,38,"Intuit to lay off 1,800 employees, plans to rehire in new AI and customer roles"


,id,title
40,40,"China is leading on GenAI experimentation, but lags U.S. in implementation, survey shows"
42,42,"China leads the world in adoption of generative AI, survey shows"
45,45,Chinas AI price war and Japanese who dont know how to use it


...

,id,title
168,168,"FTC source: Microsoft ditching its OpenAI board observer seat is unlikely to resolve concerns at the agency, which is conducting an antitrust review of AI deals"
172,172,Microsoft Quits OpenAI's Board Amid Antitrust Scrutiny
173,173,Microsoft Quits OpenAIs Board Amid Antitrust Scrutiny


,id,title
175,175,"The VC behind Mistral, Revolut and Slack just raised $2.3 billion to invest in long-term AI champions: The era of the quick buck mentality is over"
176,176,"Index Ventures, an early investor in Mistral and Cohere, closed $2.3B to chase AI breakthroughs and says that 50%+ of its recent investments have been in AI"
177,177,Index Ventures Raises $2 Billion Amid AI Platform Shift
178,178,Index Ventures raises $2.3bn to chase AI breakthroughs
179,179,"Austin-based Command Zero, which uses automation and LLMs to help with cybersecurity investigations, emerges from stealth with $21M in seed funding led by a16z"
180,180,"Hayden AI, which provides vision AI tools for cities to monitor traffic safety and more, raised a $90M Series C led by TPG's The Rise Fund"
181,181,"Sequoia, Nvidia Back Startup Fireworks AI at $552 Million Valuation"
183,183,"Captions, which uses AI to let people create and edit videos, raised $60M led by Index Ventures at a $500M valuation, taking its total funding to $100M"
184,184,"Ex-Microsoft engineers AI video startup scores $60M from top VCs, Jared Leto"


,id,title
84,84,Intel Capital backs AI construction startup that could boost Intels own manufacturing prospects
187,187,AMD buys developer Silo AI in bid to match Nvidia's product rangeFirst it comes for market leader's GPUs ... now it's nibbling at softwareAI + ML2 hrs|
188,188,"AMD agrees to buy Helsinki-based Silo AI, which offers tailored AI models and platforms to enterprise customers, for $665M in cash; Silo has a 300-person team"
189,189,AMD To Acquire Silo AI To Accelerate The Companys AI Ambitions
190,190,AMD to buy Finnish start-up Silo AI for $665mn in drive to compete with Nvidia
191,191,AMD to buy Silo AI for $665mn in drive to compete with Nvidia


,id,title
195,195,"2 Popular Artificial Intelligence (AI) Stocks to Sell Before They Plunge 64% and 67%, According to Certain Wall Street Analysts"
196,196,Citi Research analysts: Take profits in AI high-fliers (NASDAQ:AIQ)
197,197,Citi names undervalued AI stocks poised for rerating
199,199,3 Bitcoin Mining Stocks to Buy If Youre Betting Big on AI
200,200,These 3 Small-Caps Make for a Powerful Bitcoin/AI Play


,id,title
66,66,Why An Nvidia Engineer Quit at the Start of the AI Boom
202,202,"Nvidia Is Becoming the IBM of AI, Says Former Apple Engineer"
203,203,Can Nvidia Sustain Its AI-Fueled Surge as Stock Price Nears Record High?
204,204,Nvidia Is Buying Shares of These 5 AI Companies. Should You Do the Same?
207,207,$4 Trillion Appears Inevitable as Nvidia Remains the Star of the Generative AI Boom


# Save and email headlines


In [53]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.src}]({row.url})")
    html_str += f'{row.Index}.<a href="{row.url}">{row.title} - {row.src}</a><br />\n'


2024-07-11 10:31:02,906 - AInewsbot - INFO - [0. Samsung launches first-of-a-kind smart ring, watch with AI features to help monitor your health - Google News](https://nypost.com/2024/07/10/business/samsung-launches-first-of-a-kind-smart-ring-watch-with-ai-features/)
2024-07-11 10:31:02,919 - AInewsbot - INFO - [1. Samsung's new Galaxy Ring will provide users with AI-powered health insights - Google News](https://qz.com/samsung-galaxy-smart-ring-oura-ai-competitor-health-1851585334)
2024-07-11 10:31:02,922 - AInewsbot - INFO - [2. Samsung unveils its new Galaxy Ring, an AI-backed competitor to the Oura Ring - Business Insider](https://www.businessinsider.com/guides/tech/samsung-galaxy-ring-release-date-price-features-specs)
2024-07-11 10:31:02,936 - AInewsbot - INFO - [3. Samsung Unveils Exclusive Galaxy Z Flip6 Olympic Edition, Powered by Galaxy AI, for Paris 2024 Athletes - Google News](https://news.samsung.com/global/samsung-unveils-exclusive-galaxy-z-flip6-olympic-edition-powered-b

2024-07-11 10:31:02,990 - AInewsbot - INFO - [36. Tax preparation company Intuit to lay off 1800 as part of an AI-focused reorganization plan - Google News](https://apnews.com/article/intuit-layoffs-reorganization-ai-tax-prep-0fd5d4d4072fd2eb3a7bee04d820264d)
2024-07-11 10:31:02,992 - AInewsbot - INFO - [37. Intuit is laying off 1,800 employees as AI leads to a strategic shift - Reddit](https://www.reddit.com/r/technology/comments/1e09vow/intuit_is_laying_off_1800_employees_as_ai_leads/)
2024-07-11 10:31:02,993 - AInewsbot - INFO - [38. Intuit to lay off 1,800 employees, plans to rehire in new AI and customer roles - Google News](https://siliconangle.com/2024/07/10/intuit-lay-off-1800-employees-plans-rehire-new-ai-customer-roles/)
2024-07-11 10:31:02,993 - AInewsbot - INFO - [39. ByteDance, Alibaba, SenseTime top Chinas generative AI infrastructure services market - Google News](https://www.scmp.com/tech/big-tech/article/3269939/bytedance-alibaba-sensetime-lead-generative-ai-infrastruc

2024-07-11 10:31:03,010 - AInewsbot - INFO - [69. The Creator Economys AI Tailwind - Google News](https://www.theinformation.com/articles/the-creator-economys-ai-tailwind)
2024-07-11 10:31:03,011 - AInewsbot - INFO - [70. There's a reason AI firms can barely conceal their contempt for the creative industry - Business Insider](https://www.businessinsider.com/why-ai-firms-can-barely-hide-their-contempt-for-artists-2024-7)
2024-07-11 10:31:03,011 - AInewsbot - INFO - [71. Skepticism of artificial-intelligence hype is growing on Wall Street - Google News](https://www.marketwatch.com/story/wall-street-is-becoming-more-skeptical-of-artificial-intelligence-hype-helping-to-power-stocks-983aa5ca)
2024-07-11 10:31:03,011 - AInewsbot - INFO - [72. Warnings About an AI Bubble Are Growing. When Could It Burst? - Google News](https://www.inc.com/sam-blum/new-warnings-ai-bubble-when-could-it-burst.html)
2024-07-11 10:31:03,012 - AInewsbot - INFO - [73. Get Ready for More AI Mania This Earnings Season

2024-07-11 10:31:03,020 - AInewsbot - INFO - [104. Microsoft's AI Speech Generator Is Too Realistic to Release, Researchers Say - Google News](https://www.extremetech.com/internet/microsofts-ai-speech-generator-is-too-realistic-to-release-researchers)
2024-07-11 10:31:03,020 - AInewsbot - INFO - [105. Microsofts AI Voice Cloning Tech Is So Good, You Cant Use It - Google News](https://decrypt.co/238419/microsoft-ai-voice-clone-human-parity)
2024-07-11 10:31:03,020 - AInewsbot - INFO - [106. AI speech clone is so real that makers say its potential risks could prove too dangerous - Google News](https://nypost.com/2024/07/10/lifestyle/ai-speech-clone-is-so-real-that-makers-say-its-potential-risks-could-prove-too-dangerous/)
2024-07-11 10:31:03,020 - AInewsbot - INFO - [107. AI speech generator 'reaches human parity'  but it's too dangerous to release, scientists say - Google News](https://www.livescience.com/technology/artificial-intelligence/ai-speech-generator-reaches-human-parity-but-it

2024-07-11 10:31:03,024 - AInewsbot - INFO - [137. AI: Sleep Computational Neuroscience, Dreams, Loneliness, and Predictive Coding - HackerNoon](https://hackernoon.com/ai-sleep-computational-neuroscience-dreams-loneliness-and-predictive-coding)
2024-07-11 10:31:03,024 - AInewsbot - INFO - [138. AIs understanding and reasoning skills cant be assessed by current tests - Google News](https://www.sciencenews.org/article/ai-understanding-reasoning-skill-assess)
2024-07-11 10:31:03,024 - AInewsbot - INFO - [139. Could AIs become conscious? Right now, we have no way to tell. - Ars Technica](https://arstechnica.com/science/2024/07/could-ais-become-conscious-right-now-we-have-no-way-to-tell/)
2024-07-11 10:31:03,024 - AInewsbot - INFO - [140. Solving everything wrong with Large Language Models - Google News](https://www.tlnt.com/articles/solving-everything-wrong-with-large-language-models)
2024-07-11 10:31:03,024 - AInewsbot - INFO - [141. Whats next in on-device generative AI? - Google News](h

2024-07-11 10:31:03,029 - AInewsbot - INFO - [171. Microsoft Surrenders OpenAI Board Position - NYT Tech](https://www.nytimes.com/2024/07/10/technology/microsoft-openai-board.html)
2024-07-11 10:31:03,029 - AInewsbot - INFO - [172. Microsoft Quits OpenAI's Board Amid Antitrust Scrutiny - Hacker News 2](https://www.wsj.com/tech/ai/microsoft-withdraws-from-openais-board-amid-antitrust-scrutiny-aab6ff1e)
2024-07-11 10:31:03,029 - AInewsbot - INFO - [173. Microsoft Quits OpenAIs Board Amid Antitrust Scrutiny - Google News](https://www.wsj.com/tech/ai/microsoft-withdraws-from-openais-board-amid-antitrust-scrutiny-aab6ff1e)
2024-07-11 10:31:03,029 - AInewsbot - INFO - [174. Iconic AI, a London-based game development startup, just raised $4 million using this 10-slide pitch deck - Business Insider](https://www.businessinsider.com/iconic-ai-video-game-development-startup-raises-pre-seed-funding-2024-7)
2024-07-11 10:31:03,029 - AInewsbot - INFO - [175. The VC behind Mistral, Revolut and Slack 

2024-07-11 10:31:03,033 - AInewsbot - INFO - [204. Nvidia Is Buying Shares of These 5 AI Companies. Should You Do the Same? - Google News](https://www.fool.com/investing/2024/07/10/nvidia-is-buying-shares-of-these-5-ai-companies/)
2024-07-11 10:31:03,033 - AInewsbot - INFO - [205. AI frenzy propels Taiwan's TSMC to trillion dollar club - Google News](https://www.reuters.com/technology/artificial-intelligence/ai-frenzy-takes-taiwans-tsmc-record-peak-puts-it-trillion-dollar-club-2024-07-11/)
2024-07-11 10:31:03,033 - AInewsbot - INFO - [206. The AI chip race is minting thousands of new millionaires in Taiwan, but not everyone is benefiting - Business Insider](https://www.businessinsider.com/taiwan-new-millionaires-tsmc-ai-semiconductor-chip-ubs-2024-7)
2024-07-11 10:31:03,034 - AInewsbot - INFO - [207. $4 Trillion Appears Inevitable as Nvidia Remains the Star of the Generative AI Boom - HackerNoon](https://hackernoon.com/$4-trillion-appears-inevitable-as-nvidia-remains-the-star-of-the-ge

In [54]:
# save headlines
with open('headlines.html', 'w') as f:
    f.write(html_str)


In [55]:
# send mail
log("Sending headlines email")
subject = f'AI headlines {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-11 10:31:03,039 - AInewsbot - INFO - Sending headlines email


# Save individual pages 

In [56]:
# fetch pages
# Create a queue for multiprocessing and populate it 
log("Queuing URLs for scraping")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.url, row.title))
    

2024-07-11 10:31:04,510 - AInewsbot - INFO - Queuing URLs for scraping


In [57]:
# scrape urls in queue asynchronously
num_browsers = 4

callable = process_url_queue_factory(queue)

log(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
saved_pages = launch_drivers(num_browsers, callable)


2024-07-11 10:31:04,525 - AInewsbot - INFO - fetching 212 pages using 4 browsers
2024-07-11 10:31:04,526 - AInewsbot - INFO - get_driver - 32228 Initializing webdriver
2024-07-11 10:31:04,527 - AInewsbot - INFO - get_driver - 32228 Initializing webdriver
2024-07-11 10:31:04,527 - AInewsbot - INFO - get_driver - 32228 Initializing webdriver
2024-07-11 10:31:04,527 - AInewsbot - INFO - get_driver - 32228 Initializing webdriver
2024-07-11 10:31:24,648 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-11 10:31:24,648 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-11 10:31:24,649 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-11 10:31:24,650 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-11 10:31:24,650 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-11 10:31:24,650 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-11 10:31:24,653 - AInewsbot - INFO 

2024-07-11 10:33:07,083 - AInewsbot - INFO - Processing https://www.businessinsider.com/elon-musk-neuralink-update-what-to-expect-2024-7
2024-07-11 10:33:07,083 - AInewsbot - INFO - get_url(https://www.businessinsider.com/elon-musk-neuralink-update-what-to-expect-2024-7) - starting get_url https://www.businessinsider.com/elon-musk-neuralink-update-what-to-expect-2024-7
2024-07-11 10:33:14,456 - AInewsbot - INFO - get_url(Analysis | Tech winners from Trumps 2024 platform: crypto, AI and Elon Musk) - Saving Analysis___Tech_winners_from_Trumps_2024_platform__crypto__AI_and_Elon_Musk_20240711_103314.html as utf-8
2024-07-11 10:33:14,457 - AInewsbot - INFO - Processing https://www.ft.com/content/8266a1a5-a785-4ad8-bfa1-d6a606704f3a
2024-07-11 10:33:14,457 - AInewsbot - INFO - get_url(https://www.ft.com/content/8266a1a5-a785-4ad8-bfa1-d6a606704f3a) - starting get_url https://www.ft.com/content/8266a1a5-a785-4ad8-bfa1-d6a606704f3a
2024-07-11 10:33:14,491 - AInewsbot - INFO - get_url(Tech winn

2024-07-11 10:33:48,268 - AInewsbot - INFO - Processing https://www.businessinsider.com/tiktok-music-ai-chatbot-tonik-could-help-take-on-spotify-2024-7
2024-07-11 10:33:48,268 - AInewsbot - INFO - get_url(https://www.businessinsider.com/tiktok-music-ai-chatbot-tonik-could-help-take-on-spotify-2024-7) - starting get_url https://www.businessinsider.com/tiktok-music-ai-chatbot-tonik-could-help-take-on-spotify-2024-7
2024-07-11 10:33:49,659 - AInewsbot - INFO - get_url(Here's every AI feature coming to Samsung Galaxy foldable phones) - Saving Here_s_every_AI_feature_coming_to_Samsung_Galaxy_foldable_phones_20240711_103349.html as utf-8
2024-07-11 10:33:49,660 - AInewsbot - INFO - Processing https://www.bbc.com/news/articles/c4ngr3r0914o
2024-07-11 10:33:49,660 - AInewsbot - INFO - get_url(https://www.bbc.com/news/articles/c4ngr3r0914o) - starting get_url https://www.bbc.com/news/articles/c4ngr3r0914o
2024-07-11 10:33:52,366 - AInewsbot - INFO - get_url(Fanuc opens $110m automation campus i

2024-07-11 10:34:25,319 - AInewsbot - INFO - get_url(Early Apple tech bloggers are shocked to find their name and work have been AI-zombified) - Saving Early_Apple_tech_bloggers_are_shocked_to_find_their_name_and_work_have_been_AI-zombified_20240711_103425.html as utf-8
2024-07-11 10:34:25,320 - AInewsbot - INFO - Processing https://siliconangle.com/2024/07/10/intuit-lay-off-1800-employees-plans-rehire-new-ai-customer-roles/
2024-07-11 10:34:25,320 - AInewsbot - INFO - get_url(https://siliconangle.com/2024/07/10/intuit-lay-off-1800-employees-plans-rehire-new-ai-customer-roles/) - starting get_url https://siliconangle.com/2024/07/10/intuit-lay-off-1800-employees-plans-rehire-new-ai-customer-roles/
2024-07-11 10:34:30,647 - AInewsbot - INFO - get_url(Intuit decimates staff, hopes to hire same number in AI refocusWhat's that Lassie? The IRS hopes to offer free direct tax filing for everyone next year?Applications19 hrs|6) - Saving Intuit_decimates_staff__hopes_to_hire_same_number_in_AI_re

2024-07-11 10:34:54,874 - AInewsbot - INFO - get_url(Chinas AI price war and Japanese who dont know how to use it) - Saving Chinas_AI_price_war_and_Japanese_who_dont_know_how_to_use_it_20240711_103454.html as utf-8
2024-07-11 10:34:54,875 - AInewsbot - INFO - Processing https://www.forbes.com/sites/bernardmarr/2024/07/10/how-bcg-is-revolutionizing-consulting-with-ai-a-case-study/
2024-07-11 10:34:54,875 - AInewsbot - INFO - get_url(https://www.forbes.com/sites/bernardmarr/2024/07/10/how-bcg-is-revolutionizing-consulting-with-ai-a-case-study/) - starting get_url https://www.forbes.com/sites/bernardmarr/2024/07/10/how-bcg-is-revolutionizing-consulting-with-ai-a-case-study/
2024-07-11 10:34:58,679 - AInewsbot - INFO - get_url(Japan Enhances AI Sovereignty With Advanced ABCI 3.0 Supercomputer) - Saving Japan_Enhances_AI_Sovereignty_With_Advanced_ABCI_3_0_Supercomputer_20240711_103458.html as utf-8
2024-07-11 10:34:58,682 - AInewsbot - INFO - Processing https://hackernoon.com/cybersecurity-

...

2024-07-11 10:39:59,548 - AInewsbot - INFO - get_url(https://www.gadgets360.com/ai/news/odyssey-ai-text-to-video-model-hollywood-grade-report-6067589) - starting get_url https://www.gadgets360.com/ai/news/odyssey-ai-text-to-video-model-hollywood-grade-report-6067589
2024-07-11 10:40:00,764 - AInewsbot - INFO - get_url(The Washington Post made an AI chatbot for questions about climate) - Saving The_Washington_Post_made_an_AI_chatbot_for_questions_about_climate_20240711_104000.html as utf-8
2024-07-11 10:40:00,766 - AInewsbot - INFO - Processing https://www.businessinsider.com/omnicom-launches-artbotai-generative-ai-ad-creation-platform-2024-7
2024-07-11 10:40:00,766 - AInewsbot - INFO - get_url(https://www.businessinsider.com/omnicom-launches-artbotai-generative-ai-ad-creation-platform-2024-7) - starting get_url https://www.businessinsider.com/omnicom-launches-artbotai-generative-ai-ad-creation-platform-2024-7
2024-07-11 10:40:07,545 - AInewsbot - INFO - get_url(SoundHound Chat AI Launc

2024-07-11 10:40:32,965 - AInewsbot - INFO - Processing https://www.wsj.com/tech/ai/microsoft-withdraws-from-openais-board-amid-antitrust-scrutiny-aab6ff1e
2024-07-11 10:40:32,965 - AInewsbot - INFO - get_url(https://www.wsj.com/tech/ai/microsoft-withdraws-from-openais-board-amid-antitrust-scrutiny-aab6ff1e) - starting get_url https://www.wsj.com/tech/ai/microsoft-withdraws-from-openais-board-amid-antitrust-scrutiny-aab6ff1e
2024-07-11 10:40:40,221 - AInewsbot - INFO - get_url(Microsoft exits OpenAI's boardroom to sidestep regulatory scrutinyRedmond 'confident in the company's direction' says withdrawal letterAI + ML24 hrs|8) - Saving Microsoft_exits_OpenAI_s_boardroom_to_sidestep_regulatory_scrutinyRedmond__confident_in_the_company_s_direction__says_withdrawal_letterAI___ML24_hrs_8_20240711_104040.html as utf-8
2024-07-11 10:40:40,228 - AInewsbot - INFO - Processing https://www.wsj.com/tech/ai/microsoft-withdraws-from-openais-board-amid-antitrust-scrutiny-aab6ff1e
2024-07-11 10:40:40,

2024-07-11 10:41:05,395 - AInewsbot - INFO - get_url(Hayden AI, which provides vision AI tools for cities to monitor traffic safety and more, raised a $90M Series C led by TPG's The Rise Fund) - Saving Hayden_AI__which_provides_vision_AI_tools_for_cities_to_monitor_traffic_safety_and_more__raised_a__90M_Series_C_led_by_TPG_s_The_Rise_Fund_20240711_104105.html as utf-8
2024-07-11 10:41:05,396 - AInewsbot - INFO - Processing https://venturebeat.com/ai/ex-microsoft-engineers-ai-video-startup-scores-60m-from-top-vcs-jared-leto/
2024-07-11 10:41:05,397 - AInewsbot - INFO - get_url(https://venturebeat.com/ai/ex-microsoft-engineers-ai-video-startup-scores-60m-from-top-vcs-jared-leto/) - starting get_url https://venturebeat.com/ai/ex-microsoft-engineers-ai-video-startup-scores-60m-from-top-vcs-jared-leto/
2024-07-11 10:41:12,762 - AInewsbot - INFO - get_url(Skild AI, which is building a foundational model for robotics, raised a $300M Series A at a $1.5B valuation led by Lightspeed, SoftBank, C

2024-07-11 10:41:38,641 - AInewsbot - INFO - Processing https://seekingalpha.com/news/4122982-citi-research-analysts-take-profits-in-ai-high-fliers
2024-07-11 10:41:38,641 - AInewsbot - INFO - get_url(https://seekingalpha.com/news/4122982-citi-research-analysts-take-profits-in-ai-high-fliers) - starting get_url https://seekingalpha.com/news/4122982-citi-research-analysts-take-profits-in-ai-high-fliers
2024-07-11 10:41:45,617 - AInewsbot - INFO - get_url(Palantir Rockets, Nvidia Rises As This AI Peer Orbits Buy Point) - Saving Palantir_Rockets__Nvidia_Rises_As_This_AI_Peer_Orbits_Buy_Point_20240711_104145.html as utf-8
2024-07-11 10:41:45,621 - AInewsbot - INFO - Processing https://www.cnbc.com/2024/07/11/citi-names-undervalued-ai-stocks-poised-for-rerating.html
2024-07-11 10:41:45,621 - AInewsbot - INFO - get_url(https://www.cnbc.com/2024/07/11/citi-names-undervalued-ai-stocks-poised-for-rerating.html) - starting get_url https://www.cnbc.com/2024/07/11/citi-names-undervalued-ai-stocks-

2024-07-11 10:42:10,925 - AInewsbot - INFO - Processing https://www.datacenterdynamics.com/en/news/goldman-sachs-1tn-to-be-spent-on-ai-data-centers-chips-and-utility-upgrades-with-little-to-show-for-it-so-far/
2024-07-11 10:42:10,926 - AInewsbot - INFO - get_url(https://www.datacenterdynamics.com/en/news/goldman-sachs-1tn-to-be-spent-on-ai-data-centers-chips-and-utility-upgrades-with-little-to-show-for-it-so-far/) - starting get_url https://www.datacenterdynamics.com/en/news/goldman-sachs-1tn-to-be-spent-on-ai-data-centers-chips-and-utility-upgrades-with-little-to-show-for-it-so-far/
2024-07-11 10:42:18,413 - AInewsbot - INFO - get_url(The AI chip race is minting thousands of new millionaires in Taiwan, but not everyone is benefiting) - Saving The_AI_chip_race_is_minting_thousands_of_new_millionaires_in_Taiwan__but_not_everyone_is_benefiting_20240711_104218.html as utf-8
2024-07-11 10:42:18,415 - AInewsbot - INFO - Processing https://www.theregister.com/2024/07/10/cyrusone_ai_datacente

In [58]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'url', 'title', 'path']
pages_df

,id,url,title,path
0,0,https://nypost.com/2024/07/10/business/samsung...,"Samsung launches first-of-a-kind smart ring, w...",htmldata/Samsung_launches_first-of-a-kind_smar...
1,6,https://arstechnica.com/gadgets/2024/07/the-ga...,"Galaxy Z Fold & Z Flip 6, Watch Ultra, and new...",htmldata/Galaxy_Z_Fold___Z_Flip_6__Watch_Ultra...
2,10,https://www.bloomberg.com/news/articles/2024-0...,Elon Musk Says Second Neuralink Brain Implant ...,htmldata/Elon_Musk_Says_Second_Neuralink_Brain...
3,14,https://www.nytimes.com/2024/07/11/business/bi...,Biden Offers $1.7 Billion to Help Factories Bu...,htmldata/Biden_Offers__1_7_Billion_to_Help_Fac...
4,18,https://www.businessinsider.com/apple-betting-...,Apple seems super confident AI will make you w...,htmldata/Apple_seems_super_confident_AI_will_m...
...,...,...,...,...
207,194,https://www.thestreet.com/investing/stocks/ana...,Analysts reboot Apple stock price target on AI...,htmldata/Analysts_reboot_Apple_stock_price_tar...
208,198,https://www.marketwatch.com/story/these-bitcoi...,AI is driving these bitcoin miners shares high...,htmldata/AI_is_driving_these_bitcoin_miners_sh...
209,202,https://www.entrepreneur.com/business-news/nvi...,"Nvidia Is Becoming the IBM of AI, Says Former ...",htmldata/Nvidia_Is_Becoming_the_IBM_of_AI__Say...
210,205,https://www.reuters.com/technology/artificial-...,AI frenzy propels Taiwan's TSMC to trillion do...,htmldata/AI_frenzy_propels_Taiwan_s_TSMC_to_tr...


In [59]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [60]:
AIdf

,id,src,title,url,isAI,date,topic_str,title_topic_str,cluster,path
0,0,Google News,"Samsung launches first-of-a-kind smart ring, w...",https://nypost.com/2024/07/10/business/samsung...,True,2024-07-11,"AI Features, Big Tech, Health Monitoring, Heal...","Samsung launches first-of-a-kind smart ring, w...",0,htmldata/Samsung_launches_first-of-a-kind_smar...
1,1,Google News,Samsung's new Galaxy Ring will provide users w...,https://qz.com/samsung-galaxy-smart-ring-oura-...,True,2024-07-11,"AI-Powered Health Insights, Big Tech, Galaxy R...",Samsung's new Galaxy Ring will provide users w...,0,htmldata/Samsung_s_new_Galaxy_Ring_will_provid...
2,2,Business Insider,"Samsung unveils its new Galaxy Ring, an AI-bac...",https://www.businessinsider.com/guides/tech/sa...,True,2024-07-11,"Big Tech, Galaxy Ring, Oura Ring, Samsung","Samsung unveils its new Galaxy Ring, an AI-bac...",0,htmldata/Samsung_unveils_its_new_Galaxy_Ring__...
3,3,Google News,Samsung Unveils Exclusive Galaxy Z Flip6 Olymp...,https://news.samsung.com/global/samsung-unveil...,True,2024-07-11,"Athletes, Big Tech, Galaxy AI, Galaxy Z Flip6,...",Samsung Unveils Exclusive Galaxy Z Flip6 Olymp...,0,htmldata/Samsung_Unveils_Exclusive_Galaxy_Z_Fl...
4,4,Business Insider,"Samsung announces its latest smartwatches, inc...",https://www.businessinsider.com/guides/tech/sa...,True,2024-07-11,"Apple, Big Tech, Galaxy Watch Ultra, Samsung, ...","Samsung announces its latest smartwatches, inc...",0,htmldata/Samsung_announces_its_latest_smartwat...
...,...,...,...,...,...,...,...,...,...,...
207,207,HackerNoon,$4 Trillion Appears Inevitable as Nvidia Remai...,https://hackernoon.com/$4-trillion-appears-ine...,True,2024-07-11,"AI Economic Impacts, Gen AI, Generative AI, Nv...",$4 Trillion Appears Inevitable as Nvidia Remai...,17,htmldata/_4_Trillion_Appears_Inevitable_as_Nvi...
208,208,Google News,Goldman Sachs: $1tn to be spent on AI data cen...,https://www.datacenterdynamics.com/en/news/gol...,True,2024-07-11,"AI Economic Impacts, Big Tech, Chips, Data Cen...",Goldman Sachs: $1tn to be spent on AI data cen...,-1,htmldata/Goldman_Sachs___1tn_to_be_spent_on_AI...
209,209,The Register,CyrusOne scores another $7.9B in debt financin...,https://www.theregister.com/2024/07/10/cyruson...,True,2024-07-11,"Cyrusone, Datacenter, Debt Financing, Expansio...",CyrusOne scores another $7.9B in debt financin...,-1,htmldata/CyrusOne_scores_another__7_9B_in_debt...
210,210,Google News,Oracle Stock Falls On Report That Talks Are Of...,https://www.investors.com/news/technology/orac...,True,2024-07-11,"AI Startup, Big Tech, Cloud Deal, Copilot, Elo...",Oracle Stock Falls On Report That Talks Are Of...,-1,htmldata/Oracle_Stock_Falls_On_Report_That_Tal...


# Summarize individual pages

In [61]:
print(SUMMARIZE_SYSTEM_PROMPT)


You are a summarization assistant.
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will focus on the top 3 points of the text and keep the response concise."
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will output Markdown format.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.



In [62]:
print(SUMMARIZE_USER_PROMPT)


Summarize the main points of the following text concisely in at most 3 bullet points:



In [72]:
# Here we are fetching all at once, could be 200 summaries, so we are firing off 200 REST requests at once
# This seems like a bad idea, could loop through and fire off e.g. 10 at a time, or use queues and workers (seems pointless)
# But it works and runs fast on 3.5 and if ChatGPT doesn't like it they could throttle it

log("Starting summarize")
responses = await fetch_all_summaries(AIdf)
log(f"Received {len(responses)} summaries")
print(responses[0])


2024-07-11 10:58:39,283 - AInewsbot - INFO - Starting summarize
2024-07-11 10:58:39,334 - AInewsbot - INFO - fetch_all_summaries - Page title: Samsung launches first-of-a-kind smart ring, watch with AI features
Social card title: Samsung launches first-of-a-kind smart ring, watch with AI features to help monitor your health
Social card description: The AI deployment on Samsung’s new premium watch and ring is narrowly focused on improving and maintaining personal health — the main reason most people buy wearable technology.

2024-07-11 10:58:39,404 - AInewsbot - INFO - fetch_all_summaries - Page title: Samsung launched the Galaxy Ring, an AI-powered competitor to Oura smart rings
Social card title: Samsung's new Galaxy Ring will provide users with AI-powered health insights
Social card description: Samsung just launched the Galaxy Ring, an AI-powered competitor to Oura smart rings

2024-07-11 10:58:39,572 - AInewsbot - INFO - fetch_all_summaries - Page title: Samsung Galaxy Ring: Releas

2024-07-11 10:58:40,725 - AInewsbot - INFO - fetch_all_summaries - Page title: Samsung, Google and Apple make AI features selling point in phones
Social card title: Samsung's new lineup shows how AI is driving smartphone growth
Social card description: The giant phone maker's new feature push will soon be echoed in Google and Apple launches.

2024-07-11 10:58:40,781 - AInewsbot - INFO - fetch_all_summaries - Page title: Here's every AI feature coming to Samsung Galaxy foldable phones | ZDNET
Social card title: Here's every AI feature coming to Samsung Galaxy foldable phones
Social card description: At Samsung Unpacked, the company unveiled the Galaxy Z Flip 6 and Galaxy Z Fold 6, both of which are fitted with new AI features.

2024-07-11 10:58:40,843 - AInewsbot - INFO - fetch_all_summaries - Page title: Samsung launches foldable phones with enhanced AI to stay ahead of Chinese rivals

2024-07-11 10:58:40,906 - AInewsbot - INFO - fetch_all_summaries - Page title: TikTok's Chatbot Shows

2024-07-11 10:58:42,347 - AInewsbot - INFO - fetch_all_summaries - Page title: China’s homebrew openKylinOS creates a cut for AI PCs • The Register
Social card title: China’s homebrew openKylinOS creates a cut for AI PCs
Social card description: Devs of OS named for a mythical beast join in the 'local models will will deliver legendary productivity' trope

2024-07-11 10:58:42,376 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI’s China Ban Doesn’t Apply to Microsoft’s Azure China — The Information
Social card title: OpenAI’s China Ban Doesn’t Apply to Microsoft’s Azure China
Social card description: OpenAI last month announced plans to crack down on China-based artificial intelligence developers covertly accessing the startup’s technology, which it doesn’t offer in China. But OpenAI’s conversational AI models are available to Chinese businesses—if they sign up for Microsoft’s Azure cloud ...

2024-07-11 10:58:42,422 - AInewsbot - INFO - fetch_all_summaries - Page title: Ch

2024-07-11 10:58:43,235 - AInewsbot - INFO - fetch_all_summaries - Page title: AI-automated ammo vending machines may come to Florida grocery stores soon | WFLA
Social card title: AI-automated ammo vending machines may come to Florida grocery stores soon
Social card description: Vending machines can have a variety of goods like chips and drinks, but have you ever heard of a vending machine that dispenses ammunition?

2024-07-11 10:58:43,288 - AInewsbot - INFO - fetch_all_summaries - Page title: eGrowcery launches AI recipes | Supermarket News
Social card title: eGrowcery launches AI recipes
Social card description: Grocers can customize new tool based on shopper preference and inventory 

2024-07-11 10:58:43,315 - AInewsbot - INFO - fetch_all_summaries - Page title: Ex-OpenAI Employee  Compares Firm's Work to 'Building the Titanic' - Business Insider
Social card title: A former OpenAI safety employee said he quit because the company's leaders were 'building the Titanic' and wanted 'new

2024-07-11 10:58:44,767 - AInewsbot - INFO - fetch_all_summaries - Page title: Artificial intelligence helps robot supplier Fanuc expand in Michigan
Social card title: Artificial intelligence helps robot supplier Fanuc expand in Oakland County
Social card description: Japanese supplier Fanuc Corp. on Wednesday held the grand opening of a $110 million, 650,000-square-foot manufacturing site in Auburn Hills.

2024-07-11 10:58:44,782 - AInewsbot - INFO - fetch_all_summaries - Page title: Aitomatic's SemiKong uses AI to reshape chipmaking processes | VentureBeat
Social card title: Aitomatic’s SemiKong uses AI to reshape chipmaking processes
Social card description: Aitomatic announced SemiKong, the world's first open-source AI Large Language Model (LLM) designed specifically for the chip industry.

2024-07-11 10:58:44,806 - AInewsbot - INFO - fetch_all_summaries - Page title: Intel Capital backs AI construction startup that could boost Intel’s own manufacturing prospects  | TechCrunch
Soci

...

2024-07-11 10:58:48,760 - AInewsbot - INFO - fetch_all_summaries - Page title: Generative AI updates from AWS Summit New York 2024
Social card title: Generative AI updates from AWS Summit New York 2024
Social card description: AI offerings for all skill levels, plus a free training milestone for cloud, and generative AI. Here’s all the news announced from AWS Summit in New York.

2024-07-11 10:58:48,802 - AInewsbot - INFO - fetch_all_summaries - Page title: Sensing new opportunities with AI | World Grain
Social card title: Sensing new opportunities with AI
Social card description: Supplier: Grain industry on ‘cusp of an intelligent revolution’

2024-07-11 10:58:48,827 - AInewsbot - INFO - fetch_all_summaries - Page title: Beyond the gen AI hype: Google Cloud shares key learnings | VentureBeat
Social card title: Beyond the gen AI hype: Google Cloud shares key learnings
Social card description: Multimodal gen AI is becoming "seriously clever," serving as a "personal data sidekick" that u

2024-07-11 10:58:49,469 - AInewsbot - INFO - fetch_all_summaries - Page title: Microsoft, Apple will not join OpenAI’s board as regulatory scrutiny grows - The Washington Post
Social card title: Microsoft, Apple will not join OpenAI’s board as regulatory scrutiny grows
Social card description: Microsoft and Apple will no longer sit as observers on OpenAI’s board, as regulators ask whether tech giants hold too much sway over AI upstarts.

2024-07-11 10:58:49,514 - AInewsbot - INFO - fetch_all_summaries - Page title: Microsoft and Apple ditch OpenAI board seats amid regulatory scrutiny - The Verge
Social card title: Microsoft and Apple ditch OpenAI board seats amid regulatory scrutiny
Social card description: Regulators are looking closely at Big Tech AI deals.

2024-07-11 10:58:49,554 - AInewsbot - INFO - fetch_all_summaries - Page title: Microsoft and Apple drop OpenAI seats amid antitrust scrutiny

2024-07-11 10:58:49,629 - AInewsbot - INFO - fetch_all_summaries - Page title: Microsof

2024-07-11 10:58:50,750 - AInewsbot - INFO - fetch_all_summaries - Page title: AMD to buy Finnish start-up Silo AI for $665mn in drive to compete with Nvidia

2024-07-11 10:58:50,808 - AInewsbot - INFO - fetch_all_summaries - Page title: AMD To Acquire Silo AI To Accelerate The Company’s AI Ambitions
Social card title: AMD To Acquire Silo AI To Accelerate The Company’s AI Ambitions
Social card description: AMD and Europe's largest AI lab, Silo AI, have signed an agreement for an all-cash acquisition, which will bring 125 AI PHDs to AMD.

2024-07-11 10:58:50,853 - AInewsbot - INFO - fetch_all_summaries - Page title: AMD to buy Finnish start-up Silo AI for $665mn in drive to compete with Nvidia

2024-07-11 10:58:50,924 - AInewsbot - INFO - fetch_all_summaries - Page title: AMD to buy Finnish start-up Silo AI for $665mn in drive to compete with Nvidia

2024-07-11 10:58:50,971 - AInewsbot - INFO - fetch_all_summaries - Page title: barrons.com

2024-07-11 10:58:50,972 - trafilatura.core - W

2024-07-11 10:58:52,015 - trafilatura.core - WARNING - discarding data: None
2024-07-11 10:58:56,730 - AInewsbot - INFO - Received 212 summaries


(0, {'id': 'chatcmpl-9jpXwpJLgNWiGajpguHniV8hejV6j', 'object': 'chat.completion', 'created': 1720709932, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '- Samsung has launched a new premium smartwatch and smart ring with AI features focused on monitoring and managing personal health.\n- The wearable devices analyze biometric data to provide an "energy score" and health assessments, with the smartwatch offering additional functions like detecting sleep apnea.\n- The wearable technology market is growing, with smartwatches expected to generate billions in sales, but smart rings are still considered an emerging niche market with limited growth potential according to Gartner forecasts.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1107, 'completion_tokens': 94, 'total_tokens': 1201}, 'system_fingerprint': None})


In [73]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

212

In [74]:
markdown_str = ''

for i, row in enumerate(AIdf.itertuples()):
    mdstr = f"[{i+1}. {row.title} - {row.src}]({row.url})  \n\n {row.topic_str} \n\n{response_dict[row.id]} \n\n"
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. Samsung launches first-of-a-kind smart ring, watch with AI features to help monitor your health - Google News](https://nypost.com/2024/07/10/business/samsung-launches-first-of-a-kind-smart-ring-watch-with-ai-features/)  

 AI Features, Big Tech, Health Monitoring, Healthcare, Samsung, Smart Ring, Watch 

- Samsung has launched a new premium smartwatch and smart ring with AI features focused on monitoring and managing personal health.
- The wearable devices analyze biometric data to provide an "energy score" and health assessments, with the smartwatch offering additional functions like detecting sleep apnea.
- The wearable technology market is growing, with smartwatches expected to generate billions in sales, but smart rings are still considered an emerging niche market with limited growth potential according to Gartner forecasts. 



[2. Samsung's new Galaxy Ring will provide users with AI-powered health insights - Google News](https://qz.com/samsung-galaxy-smart-ring-oura-ai-competitor-health-1851585334)  

 AI-Powered Health Insights, Big Tech, Galaxy Ring, Gen AI, Healthcare, Samsung 

- Samsung has launched the Galaxy Ring, an AI-powered health device to compete with Oura smart rings.
- The Galaxy Ring provides users with AI-powered health insights.
- Samsung's new product aims to rival Oura smart rings with its advanced features. 



[3. Samsung unveils its new Galaxy Ring, an AI-backed competitor to the Oura Ring - Business Insider](https://www.businessinsider.com/guides/tech/samsung-galaxy-ring-release-date-price-features-specs)  

 Big Tech, Galaxy Ring, Oura Ring, Samsung 

- Samsung Galaxy Ring is a discreet health-tracking wearable for the finger, offering features like heart-rate monitoring, temperature sensors, and water resistance up to 300 feet, with a battery life of six to seven days.
- Priced at $399.99, the Galaxy Ring offers AI features like Energy Score and Wellness Tips, providing insights on daily physical and mental conditions and personalized recommendations for exercise, sleep, stress, and more. 
- The Galaxy Ring is available for pre-order and will be released on July 24, with seven sizes to choose from and a charging dock that provides additional battery life. 



[4. Samsung Unveils Exclusive Galaxy Z Flip6 Olympic Edition, Powered by Galaxy AI, for Paris 2024 Athletes - Google News](https://news.samsung.com/global/samsung-unveils-exclusive-galaxy-z-flip6-olympic-edition-powered-by-galaxy-ai-for-paris-2024-athletes)  

 Athletes, Big Tech, Galaxy AI, Galaxy Z Flip6, Olympic Edition, Paris 2024, Samsung 

- Samsung introduces the Galaxy Z Flip6 Olympic Edition exclusively designed for Paris 2024 athletes, providing advanced technology and customization.
- The Olympic Edition features Galaxy AI technology, pre-loaded exclusive services and apps, and a partnership with Berluti for a unique Flipsuit Case.
- Athletes will benefit from Galaxy AI communication features, built-in services like eSIM and 5G data, and access to special apps to enhance their experience at the Olympics. 



[5. Samsung announces its latest smartwatches, including a brand-new Galaxy Watch Ultra designed to take on Apple's high-end wearable - Business Insider](https://www.businessinsider.com/guides/tech/samsung-galaxy-watch-7-ultra-release-date-price-features-specs)  

 Apple, Big Tech, Galaxy Watch Ultra, Samsung, Smartwatches, Wearable Technology 

- Samsung announced the new Galaxy Watch lineup featuring the Galaxy Watch Ultra and Galaxy Watch 7, both available for preorder and releasing on July 24.
- Samsung integrated Galaxy AI across all models, offering features such as 'Energy Score' and 'Wellness Tips' to help users track wellness metrics and receive personalized recommendations.
- The Galaxy Watch Ultra boasts features like a Quick Button for workouts, BioActive Sensor for heart rate monitoring, dual-GPS system, water resistance, and up to 16 hours of battery life, priced at $649.99. The Galaxy Watch 7 starts at $299.99 and includes upgrades like a higher storage option, new BioActive Sensor, and Exynos W1000 processor. 



...

[208. $4 Trillion Appears Inevitable as Nvidia Remains the Star of the Generative AI Boom - HackerNoon](https://hackernoon.com/$4-trillion-appears-inevitable-as-nvidia-remains-the-star-of-the-generative-ai-boom)  

 AI Economic Impacts, Gen AI, Generative AI, Nvidia, Semiconductor Chips, Stock Market, Stocks, Technology 

- Nvidia is emerging as a frontrunner for becoming the first $4 trillion stock, fueled by the generative AI boom and impressive financial performance.
- Nvidia's position as a market leader in providing computer chips for generative AI has led to comparisons with IBM's performance and potential dominance in the AI hardware industry.
- While Nvidia faces competition from other firms like Taiwan Semiconductor Manufacturing Company and Advanced Micro Devices, its continuous investment in leading AI technology keeps it ahead in the market. 



[209. Goldman Sachs: $1tn to be spent on AI data centers, chips, and utility upgrades, with "little to show for it so far" - Google News](https://www.datacenterdynamics.com/en/news/goldman-sachs-1tn-to-be-spent-on-ai-data-centers-chips-and-utility-upgrades-with-little-to-show-for-it-so-far/)  

 AI Economic Impacts, Big Tech, Chips, Data Centers, Goldman Sachs, Semiconductor Chips, Utility Upgrades 

- $1 trillion to be spent on AI data centers, chips, and utility upgrades, but unclear on financial returns and sustainable business models
- Concerns raised by experts on the sustainability of AI investment and the impact on corporate profitability and the power grid
- AI data center growth will significantly increase electricity demand, requiring substantial investment in power infrastructure. 



[210. CyrusOne scores another $7.9B in debt financing to expand AI datacenter empireLenders bet you're willing to rent GPUsOn-Prem1 day|1 - The Register](https://www.theregister.com/2024/07/10/cyrusone_ai_datacenter/)  

 Cyrusone, Datacenter, Debt Financing, Expansion, Gen AI 

- CyrusOne secured a $7.9 billion loan for AI datacenter expansion to meet the growing demand for sustainable AI infrastructure.
- The investment aims to address the need for massive quantities of accelerators like GPUs for training large language models used in AI applications.
- The emphasis on sustainability is driven by the energy and water requirements of AI datacenters, impacting both environmental credentials and operational costs. 



[211. Oracle Stock Falls On Report That Talks Are Off For Cloud Deal With Elon Musk's AI Startup - Google News](https://www.investors.com/news/technology/oracle-stock-falls-xai-musk-deal/)  

 AI Startup, Big Tech, Cloud Deal, Copilot, Elon Musk, Openai, Oracle, Stocks 

- Oracle is no longer in talks with Elon Musk's xAI startup for an expanded cloud-computing deal, leading to a drop in Oracle's stock.
- Instead of continuing with Oracle, xAI will purchase chips from Nvidia to build its own data center for training AI models.
- Oracle's cloud infrastructure business, OCI, is a significant part of its transition to a cloud services company and was highlighted during the company's earnings call. 



[212. xAI Appears to Confirm Ended Talks With Oracle Over Expanded AI Chips Agreement - Google News](https://www.wsj.com/tech/ai/xai-appears-to-confirm-ended-talks-with-oracle-over-expanded-ai-chips-agreement-e06ade8b)  

 AI Chips, AI Economic Impacts, Big Tech, Business Negotiations, Openai, Oracle, Semiconductor Chips, Xai 

- Title: wsj.com
- Main points: The text is the title of the webpage and does not contain any other content to summarize.
 



In [75]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
# display(HTML(html_str))


In [76]:
# save bullets
with open('bullets.md', 'w') as f:
    f.write(markdown_str)


In [77]:
log("Sending bullet points email")
subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-11 10:59:28,827 - AInewsbot - INFO - Sending bullet points email


# Ask ChatGPT for top categories

In [78]:
print(TOP_CATEGORIES_PROMPT)

You will act as a research assistant identifying the top stories and topics
of today's news. I will provide a list of today's news stories about AI and summary bullet points in markdown
format. You are tasked with identifying the top 10-20 stories and topics of today's news. For each top story
or topic, you will create a short title and respond with a list of titles formatted as a JSON object.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Categories of

In [79]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TOP_CATEGORIES_PROMPT + markdown_str
              }],
    n=1,
    response_format={"type": "json_object"},
    temperature=0.5
)


2024-07-11 10:59:37,904 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [80]:
list(json.loads(response.choices[0].message.content).values())[0]

['Samsung Galaxy Ring launch',
 'Samsung Galaxy Z Flip6 Olympic Edition',
 'Samsung Galaxy Watch Ultra',
 'Samsung Galaxy Z Fold 6 and Z Flip 6',
 'Elon Musk Neuralink update',
 'Trump 2024 platform on AI and crypto',
 'Biden $1.7 billion EV funding',
 'FANUC automation facility openings',
 'Apple AI-driven iPhone sales',
 'Intuit AI-focused layoffs',
 'China leads in GenAI adoption',
 'Microsoft and Apple OpenAI board exit',
 'Neuralink brain implant progress',
 'Nvidia AI market dominance',
 'NATO revised AI strategy',
 'Hayden AI $90M Series C funding',
 'AMD acquisition of Silo AI',
 'Generative AI in K-pop',
 'AI in agriculture',
 'AI disinformation campaigns']

# Final Summary

In [81]:
markdown_str = ''
for i, row in enumerate(AIdf.itertuples()):
    mdstr = f"[{i+1}. {row.title} - {row.src}]({row.url})  \n\n"
    if 0 < len(catdict[row.id]) < 11 :
        topicstr = ", ".join(catdict[row.id])
        mdstr += f"Topics: {topicstr}\n\n"
    mdstr += f"{response_dict[row.id]} \n\n"
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. Samsung launches first-of-a-kind smart ring, watch with AI features to help monitor your health - Google News](https://nypost.com/2024/07/10/business/samsung-launches-first-of-a-kind-smart-ring-watch-with-ai-features/)  

Topics: AI economic impacts, Gen AI, Nvidia, Semiconductor chips, Stocks

- Samsung has launched a new premium smartwatch and smart ring with AI features focused on monitoring and managing personal health.
- The wearable devices analyze biometric data to provide an "energy score" and health assessments, with the smartwatch offering additional functions like detecting sleep apnea.
- The wearable technology market is growing, with smartwatches expected to generate billions in sales, but smart rings are still considered an emerging niche market with limited growth potential according to Gartner forecasts. 



[2. Samsung's new Galaxy Ring will provide users with AI-powered health insights - Google News](https://qz.com/samsung-galaxy-smart-ring-oura-ai-competitor-health-1851585334)  

Topics: Stocks

- Samsung has launched the Galaxy Ring, an AI-powered health device to compete with Oura smart rings.
- The Galaxy Ring provides users with AI-powered health insights.
- Samsung's new product aims to rival Oura smart rings with its advanced features. 



[3. Samsung unveils its new Galaxy Ring, an AI-backed competitor to the Oura Ring - Business Insider](https://www.businessinsider.com/guides/tech/samsung-galaxy-ring-release-date-price-features-specs)  

- Samsung Galaxy Ring is a discreet health-tracking wearable for the finger, offering features like heart-rate monitoring, temperature sensors, and water resistance up to 300 feet, with a battery life of six to seven days.
- Priced at $399.99, the Galaxy Ring offers AI features like Energy Score and Wellness Tips, providing insights on daily physical and mental conditions and personalized recommendations for exercise, sleep, stress, and more. 
- The Galaxy Ring is available for pre-order and will be released on July 24, with seven sizes to choose from and a charging dock that provides additional battery life. 



[4. Samsung Unveils Exclusive Galaxy Z Flip6 Olympic Edition, Powered by Galaxy AI, for Paris 2024 Athletes - Google News](https://news.samsung.com/global/samsung-unveils-exclusive-galaxy-z-flip6-olympic-edition-powered-by-galaxy-ai-for-paris-2024-athletes)  

Topics: Education, Ethical issues, Gen AI, Language Models, Robots

- Samsung introduces the Galaxy Z Flip6 Olympic Edition exclusively designed for Paris 2024 athletes, providing advanced technology and customization.
- The Olympic Edition features Galaxy AI technology, pre-loaded exclusive services and apps, and a partnership with Berluti for a unique Flipsuit Case.
- Athletes will benefit from Galaxy AI communication features, built-in services like eSIM and 5G data, and access to special apps to enhance their experience at the Olympics. 



[5. Samsung announces its latest smartwatches, including a brand-new Galaxy Watch Ultra designed to take on Apple's high-end wearable - Business Insider](https://www.businessinsider.com/guides/tech/samsung-galaxy-watch-7-ultra-release-date-price-features-specs)  

Topics: AI doom, AI economic impacts, Sustainability

- Samsung announced the new Galaxy Watch lineup featuring the Galaxy Watch Ultra and Galaxy Watch 7, both available for preorder and releasing on July 24.
- Samsung integrated Galaxy AI across all models, offering features such as 'Energy Score' and 'Wellness Tips' to help users track wellness metrics and receive personalized recommendations.
- The Galaxy Watch Ultra boasts features like a Quick Button for workouts, BioActive Sensor for heart rate monitoring, dual-GPS system, water resistance, and up to 16 hours of battery life, priced at $649.99. The Galaxy Watch 7 starts at $299.99 and includes upgrades like a higher storage option, new BioActive Sensor, and Exynos W1000 processor. 



...

[208. $4 Trillion Appears Inevitable as Nvidia Remains the Star of the Generative AI Boom - HackerNoon](https://hackernoon.com/$4-trillion-appears-inevitable-as-nvidia-remains-the-star-of-the-generative-ai-boom)  

Topics: Ethical issues, Gen AI

- Nvidia is emerging as a frontrunner for becoming the first $4 trillion stock, fueled by the generative AI boom and impressive financial performance.
- Nvidia's position as a market leader in providing computer chips for generative AI has led to comparisons with IBM's performance and potential dominance in the AI hardware industry.
- While Nvidia faces competition from other firms like Taiwan Semiconductor Manufacturing Company and Advanced Micro Devices, its continuous investment in leading AI technology keeps it ahead in the market. 



[209. Goldman Sachs: $1tn to be spent on AI data centers, chips, and utility upgrades, with "little to show for it so far" - Google News](https://www.datacenterdynamics.com/en/news/goldman-sachs-1tn-to-be-spent-on-ai-data-centers-chips-and-utility-upgrades-with-little-to-show-for-it-so-far/)  

Topics: Big Tech, ChatGPT, Cohere, Gen AI, LLMs, Language Models, OpenAI, Robots

- $1 trillion to be spent on AI data centers, chips, and utility upgrades, but unclear on financial returns and sustainable business models
- Concerns raised by experts on the sustainability of AI investment and the impact on corporate profitability and the power grid
- AI data center growth will significantly increase electricity demand, requiring substantial investment in power infrastructure. 



[210. CyrusOne scores another $7.9B in debt financing to expand AI datacenter empireLenders bet you're willing to rent GPUsOn-Prem1 day|1 - The Register](https://www.theregister.com/2024/07/10/cyrusone_ai_datacenter/)  

Topics: Ethical issues, Legal issues, Policy and regulation

- CyrusOne secured a $7.9 billion loan for AI datacenter expansion to meet the growing demand for sustainable AI infrastructure.
- The investment aims to address the need for massive quantities of accelerators like GPUs for training large language models used in AI applications.
- The emphasis on sustainability is driven by the energy and water requirements of AI datacenters, impacting both environmental credentials and operational costs. 



[211. Oracle Stock Falls On Report That Talks Are Off For Cloud Deal With Elon Musk's AI Startup - Google News](https://www.investors.com/news/technology/oracle-stock-falls-xai-musk-deal/)  

Topics: Gen AI

- Oracle is no longer in talks with Elon Musk's xAI startup for an expanded cloud-computing deal, leading to a drop in Oracle's stock.
- Instead of continuing with Oracle, xAI will purchase chips from Nvidia to build its own data center for training AI models.
- Oracle's cloud infrastructure business, OCI, is a significant part of its transition to a cloud services company and was highlighted during the company's earnings call. 



[212. xAI Appears to Confirm Ended Talks With Oracle Over Expanded AI Chips Agreement - Google News](https://www.wsj.com/tech/ai/xai-appears-to-confirm-ended-talks-with-oracle-over-expanded-ai-chips-agreement-e06ade8b)  

Topics: AI economic impacts, Big Tech, OpenAI, Semiconductor chips

- Title: wsj.com
- Main points: The text is the title of the webpage and does not contain any other content to summarize.
 



In [82]:
FINAL_SUMMARY_PROMPT = f"""You are a summarization assistant. I will provide a list of today's news articlds about AI
and summary bullet points in markdown format. Bullet points will have a title and URL, a list of topics discussed, 
and a bullet-point summary of the article. You are tasked with identifying and summarizing the key themes,
common facts, and recurring elements. Your goal is to create a concise summary containing about 20 of the most 
frequently mentioned topics and developments.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

AI startup funding, New AI products

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Examples of important stories:

Major investments and funding rounds
Key technological advancements or breakthroughs
Frequently mentioned companies, organizations, or figures
Notable statements by AI leaders
Any other recurring themes or notable patterns

Instructions:

Read the summary bullet points closely.
Use only information provided in them and provide the most common facts without commentary or elaboration.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Be balanced, professional, informative, providing accurate, clear, concise summaries in a respectful neutral tone.
Focus on the most common elements across the bullet points and group similar items together.
Headers must be as short and simple as possible: use "Health Care" and not "AI developments in Health Care" or "AI in Health Care"
Ensure that you provide at least one link from the provided text for each item in the summary.
You must include at least 10 and no more than 25 items in the summary.

Example Output Format:

# Today's AI News

### Security and Privacy:
- ChatGPT Mac app had a security flaw exposing user conversations in plain text. ([Macworld](https://www.macworld.com/article/2386267/chatgpt-mac-sandboxing-security-flaw-apple-intelligence.html))
- Brazil suspended Meta from using Instagram and Facebook posts for AI training over privacy concerns. ([BBC](https://www.bbc.com/news/articles/c7291l3nvwv))

### Health Care:
- AI can predict Alzheimer's disease with 70% accuracy up to seven years in advance. ([Decrypt](https://decrypt.co/238449/ai-alzheimers-detection-70-percent-accurate-study))
- New AI system detects 13 cancers with 98% accuracy, revolutionizing cancer diagnosis. ([India Express](https://news.google.com/articles/CBMiiAFodHRwczovL2luZGlhbmV4cHJl))

Bullet Points to Summarize:

"""

In [83]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": FINAL_SUMMARY_PROMPT + markdown_str
              }],
    n=1,   
    temperature=0.5
)


2024-07-11 11:00:22,965 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [84]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


# Today's AI News

### Samsung's AI Innovations:
- Samsung launched the Galaxy Ring, a wearable health-tracking device with AI-powered features to compete with Oura smart rings. ([Business Insider](https://www.businessinsider.com/guides/tech/samsung-galaxy-ring-release-date-price-features-specs))
- The Galaxy Z Fold 6 and Z Flip 6 foldable phones were introduced, featuring enhanced AI capabilities like suggested replies and improved durability. ([Ars Technica](https://arstechnica.com/gadgets/2024/07/the-galaxy-z-fold-flip-6-are-not-as-exciting-to-samsung-as-galaxy-ai/))

### AI in Health Care:
- Neuralink aims to implant its device into a second human patient within a week, targeting enhanced human capabilities through AI integration. ([Bloomberg Tech](https://www.bloomberg.com/news/articles/2024-07-10/elon-musk-says-second-neuralink-brain-implant-about-a-week-away))
- AI is increasingly integrated into agriculture programs to address challenges like labor shortages and water scarcity, with significant industry demand for tech-skilled graduates. ([Inside Higher Ed](https://www.insidehighered.com/news/tech-innovation/artificial-intelligence/2024/07/10/ai-taking-root-growing-number-agriculture))

### AI and Big Tech:
- Microsoft and Apple have stepped down from their observer roles on OpenAI's board amid regulatory scrutiny over their influence in the AI sector. ([The Verge](https://www.theverge.com/2024/7/10/24195528/microsoft-apple-openai-board-observer-seat-drop-regulator-scrutiny))
- Nvidia's dominance in the AI chip market continues to grow, with the company's stock nearing record highs due to strong demand for AI hardware. ([Investopedia](https://www.investopedia.com/nvidia-stock-price-nears-record-high-ai-8675899))

### AI in Entertainment:
- Captions, an AI video startup founded by a former Microsoft engineer, raised \$60 million in funding to expand its AI-powered creative suite for video creators. ([VentureBeat](https://venturebeat.com/ai/ex-microsoft-engineers-ai-video-startup-scores-60m-from-top-vcs-jared-leto/))
- AI-generated jokes by ChatGPT were found to be funnier than those created by humans in a University of Southern California study. ([CBS News](https://www.cbsnews.com/news/ai-generated-jokes-funnier-than-humans-university-of-southern-california-study/))

### AI and Policy:
- The Biden administration is granting \$1.7 billion to auto parts factories to produce electric vehicles and clean energy technology, aiming to create new jobs and compete globally. ([NYT Tech](https://www.nytimes.com/2024/07/11/business/biden-administration-electric-vehicles.html))
- The GOP platform for 2024 emphasizes rolling back tech regulations on AI and cryptocurrency, advocating for AI development rooted in free speech and human flourishing. ([Washington Post](https://www.washingtonpost.com/politics/2024/07/10/gop-platform-trump-tech-crypto-ai-musk/))

### AI in Cybersecurity:
- Command Zero, a startup leveraging AI for cybersecurity investigations, emerged from stealth with \$21 million in seed funding led by Andreessen Horowitz. ([Techmeme](https://www.securityweek.com/command-zero-emerges-from-stealth-mode-to-speed-up-cyber-investigations/))
- The US DOJ and its partners seized two domain names and 968 accounts on X used by Russia's RT to develop an AI-enhanced bot farm that spread disinformation. ([PCMag](https://www.pcmag.com/news/us-disrupts-russian-bots-spreading-propaganda-on-twitter))

### AI and Ethics:
- Pope Francis called for a ban on lethal autonomous weapon systems, emphasizing the importance of human control over AI in decision-making. ([Catholic News Agency](https://www.catholicnewsagency.com/news/258241/pope-francis-tells-ai-leaders-no-machine-should-ever-choose-to-take-human-life))
- OpenAI's CTO warned of the major risks presented by AI models in their ability to persuade, influence, and control people. ([Reddit](https://www.reddit.com/r/ChatGPT/comments/1e0d4kf/openai_cto_says_ai_models_pose_incredibly_scary/))

### AI in Business:
- Intuit announced plans to lay off 1,800 employees while focusing on hiring the same number for AI-related work, shifting towards an AI-driven strategy. ([AP News](https://apnews.com/article/intuit-layoffs-reorganization-ai-tax-prep-0fd5d4d4072fd2eb3a7bee04d820264d))
- Index Ventures raised \$2.3 billion to invest in long-term AI champions, focusing on industries like healthcare, hospitality, and education. ([Techmeme](https://t.co/Gabl8WwaF0))

### AI in Education:
- Morehouse College is introducing AI teaching assistants to help students with questions and lectures, aiming to provide support outside of class hours. ([Chronicle](https://www.chronicle.com/article/a-professors-digital-mini-me))

### AI in Consumer Products:
- Apple expects AI to significantly boost iPhone sales, aiming to ship 90 million units of the iPhone 16, leveraging Apple Intelligence to reverse the decline in iPhone sales. ([Business Insider](https://www.businessinsider.com/apple-betting-ai-reverse-iphone-sales-samsung-google-apple-intelligence-2024-7))

### AI and the Economy:
- Goldman Sachs predicts \$1 trillion to be spent on AI data centers, chips, and utility upgrades, raising concerns about financial returns and sustainability. ([Data Center Dynamics](https://www.datacenterdynamics.com/en/news/goldman-sachs-1tn-to-be-spent-on-ai-data-centers-chips-and-utility-upgrades-with-little-to-show-for-it-so-far/))

### AI in Robotics:
- Skild AI, developing a "general purpose brain" for robots, raised \$300 million in Series A funding to advance AI capabilities in robotics. ([Forbes](https://www.forbes.com/sites/rashishrivastava/2024/07/09/this-15-billion-ai-company-is-building-a-general-purpose-brain-for-robots/))

### AI in Agriculture:
- A farm in Manitoba is showcasing the benefits of AI and machine learning integration in agriculture, aiming to improve crop monitoring and care. ([CTV News](https://winnipeg.ctvnews.ca/manitoba-farm-paves-way-for-ai-in-agriculture-1.6956075))

### AI in Financial Services:
- Hayden AI Technologies raised \$90 million in Series C funding to enhance its vision AI platform for urban traffic safety and sustainability. ([Finsmes](https://www.finsmes.com/2024/07/hayden-ai-raises-90m-in-series-c-funding.html))

### AI in Social Media:
- TikTok introduced the AI chatbot Tonik within its TikTok Music app to help users find information about artists, generate playlists, and stay updated with music news. ([Business Insider](https://www.businessinsider.com/tiktok-music-ai-chatbot-tonik-could-help-take-on-spotify-2024-7))

### AI in Legal and Regulatory:
- GitHub Copilot is not considered to infringe on copyrights, with machine-generated code not viewed as derivative works under current copyright law. ([Hacker News](https://felixreda.eu/2021/07/github-copilot-is-not-infringing-your-copyright/))

### AI in Research:
- Scientists at the University of Cambridge developed an AI-powered atlas to understand protein behavior in human cells, potentially leading to new disease interventions. ([Phys.org](https://phys.org/news/2024-07-ai-powered-atlas-reveals-proteins.html))

### AI in Transportation:
- SoundHound Chat AI voice assistant with integrated ChatGPT is now live in Peugeot, Opel, and Vauxhall vehicles across 11 European markets. ([Business Wire](https://www.businesswire.com/news/home/20240710077406/en/SoundHound-Chat-AI-Launches-in-Peugeot-Opel-and-Vauxhall-Vehicles-Throughout-Europe))

### AI in Gaming:
- Iconic AI raised \$4 million in pre-seed funding to change AAA game development using AI technologies for animation, translation, voice generation, and testing. ([Business Insider](https://www.businessinsider.com/iconic-ai-video-game-development-startup-raises-pre-seed-funding-2024-7))

In [85]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-07-11 11:00:22,984 - AInewsbot - INFO - Sending full summary email 


In [86]:
log("Finished")


2024-07-11 11:00:24,299 - AInewsbot - INFO - Finished


20

In [87]:
redirect_dict

{'https://news.google.com/articles/CBMiUmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy8yLXBvcHVsYXItYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtYWktMDgxMjAwMDU1Lmh0bWzSAQA': 'https://finance.yahoo.com/news/2-popular-artificial-intelligence-ai-081200055.html',
 'https://news.google.com/articles/CBMiXGh0dHBzOi8vaW52ZXN0b3JwbGFjZS5jb20vMjAyNC8wNy8zLWJpdGNvaW4tbWluaW5nLXN0b2Nrcy10by1idXktaWYteW91cmUtYmV0dGluZy1iaWctb24tYWkv0gEA': 'https://investorplace.com/2024/07/3-bitcoin-mining-stocks-to-buy-if-youre-betting-big-on-ai/',
 'https://news.google.com/articles/CBMiPmh0dHBzOi8vd3d3LmNocm9uaWNsZS5jb20vYXJ0aWNsZS9hLXByb2Zlc3NvcnMtZGlnaXRhbC1taW5pLW1l0gEA': 'https://www.chronicle.com/article/a-professors-digital-mini-me',
 'https://news.google.com/articles/CBMiWmh0dHBzOi8vd3d3LndpcmVkLmNvbS9zdG9yeS9haS1lbmVyZ3ktZGVtYW5kcy13YXRlci1pbXBhY3QtaW50ZXJuZXQtaHlwZXItY29uc3VtcHRpb24tZXJhL9IBAA': 'https://www.wired.com/story/ai-energy-demands-water-impact-internet-hyper-consumption-era/',
 'https://news.google.com/article